<div style="border-radius:10px;
            background-color:#D6EAF8;
            letter-spacing:0.5px; text-align: center;">
            
<h1 style="padding: 10px 0px; color:#2980B9; font-weight: bold; font-family: Arial, sans-serif;">
Transformacija podataka</h1>
</div>

<div style="border-radius:10px;
            background-color:#D6EAF8;
            letter-spacing:0.5px; text-align: center;">
            
<h2 style="padding: 5px 0px; color:#2980B9; font-weight: bold; font-family: Arial, sans-serif;">
Prikupljanje i predobrada podataka za Mašinsko učenje</h2>
</div>

<div style="border-radius:10px;
            background-color:#D6EAF8;
            letter-spacing:0.5px; text-align: center;">
            
<h2 style="padding: 5px 0px; color:#2980B9; font-weight: bold; font-family: Arial, sans-serif;">
Učitavanje biblioteka</h2>
</div>

In [ ]:
#Ucitavanje biblioteka
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score,roc_auc_score,f1_score
from sklearn.preprocessing import  LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import RMSprop,Adam
from keras.callbacks import EarlyStopping
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier


sns.set(style="white")
sns.set(style="whitegrid", color_codes=True)

import warnings

<div style="border-radius:10px;
            background-color:#D6EAF8;
            letter-spacing:0.5px; text-align: center;">
            
<h2 style="padding: 5px 0px; color:#2980B9; font-weight: bold; font-family: Arial, sans-serif;">
Učitavanje datasetova</h2>
</div>

In [ ]:
#Ucitavanje datasetova
airlineDataset = pd.read_csv("data/airlineData.csv")
bankDataset = pd.read_csv("data/bankData.csv", sep=';')
vehiclesDataset = pd.read_csv("data/vehiclesData.csv")

<div style="border-radius:10px;
            background-color:#D6EAF8;
            letter-spacing:0.5px; text-align: center;">
            
<h2 style="padding: 5px 0px; color:#2980B9; font-weight: bold; font-family: Arial, sans-serif;">
Enkodiranje kategorickih atributa</h2>
</div>

<div style="border-radius:10px;
            background-color:#D6EAF8;
            letter-spacing:0.5px; text-align: center;">
            
<h3 style="padding: 5px 0px; color:#2980B9; font-weight: bold; font-family: Arial, sans-serif;">
Bank dataset</h3>
</div>

In [ ]:
#Enkodiranje output varijable
label_encoder = LabelEncoder()
label_encoder.fit(bankDataset["y"])
bankDataset["y"] = label_encoder.transform(bankDataset["y"])

In [ ]:
#Kopiranje originalnog u pomocne datasetove
bankDataset.dropna()
bankDataset_MixEncoding = bankDataset.copy()
bankDataset_LabelEncoding = bankDataset.copy()
bankDataset_OneHotEncoding = bankDataset.copy()

In [ ]:
checkOutliers_bank = bankDataset.select_dtypes(include=['int', 'float']).columns
checkOutliers_bank = checkOutliers_bank.drop('y')


In [ ]:
# provlacenje kategorickih atributa kroz one hot encoder
cols = ['job', 'contact', 'marital', 'poutcome', 'month', 'day_of_week']
bankDataset_MixEncoding = pd.get_dummies(data=bankDataset_MixEncoding, columns=cols)
#bankDataset_MixEncoding.head()

In [ ]:
# Label Encoder 
labels = ['housing', 'default','education', 'loan']
for label in labels:
    label_encoder = LabelEncoder()
    label_encoder.fit(bankDataset_MixEncoding[label])
    bankDataset_MixEncoding[label] = label_encoder.transform(bankDataset_MixEncoding[label])
#bankDataset_MixEncoding.head()

In [ ]:
# Label Encoder nad celim skupom
kategoricki_atributi = bankDataset.select_dtypes(include='object').columns.tolist()
for atribut in kategoricki_atributi:
    label_encoder = LabelEncoder()
    label_encoder.fit(bankDataset[atribut])
    bankDataset_LabelEncoding[atribut] = label_encoder.transform(bankDataset_LabelEncoding[atribut])
#bankDataset_LabelEncoding.head()

In [ ]:
# One Hot Encoder nad celim skupom
bankDataset_OneHotEncoding = pd.get_dummies(data=bankDataset_OneHotEncoding,columns=kategoricki_atributi)

In [ ]:
bankDataset_MixEncoding.nunique()

<div style="border-radius:10px;
            background-color:#D6EAF8;
            letter-spacing:0.5px; text-align: center;">
            
<h3 style="padding: 5px 0px; color:#2980B9; font-weight: bold; font-family: Arial, sans-serif;">
Airline dataset</h3>
</div>

In [ ]:
#Enkodiranje output varijable
airlineDataset = airlineDataset.dropna()

from sklearn.preprocessing import  LabelEncoder
label_encoder = LabelEncoder()
label_encoder.fit(airlineDataset["satisfaction"])
airlineDataset["satisfaction"] = label_encoder.transform(airlineDataset["satisfaction"])

In [ ]:
airlineDataset.drop('id',axis=1,inplace=True)

In [ ]:
#Kopiranje originalnog dataseta u posebne datasetove za enkodiranje
airlineDataset_MixEncoding = airlineDataset.copy()
airlineDataset_OneHotEncoding = airlineDataset.copy()
airlineDataset_LabelEncoding = airlineDataset.copy()

In [ ]:
checkOutliers_airline = ['Age','Flight Distance','Departure Delay in Minutes','Arrival Delay in Minutes']

Enkodiranje

In [ ]:
# Label encode the 'Class' column
class_map = {'Eco': 0, 'Eco Plus': 1, 'Business': 2}
airlineDataset_MixEncoding['Class'] = airlineDataset_MixEncoding['Class'].replace(class_map)

In [ ]:
# One hot encode other atributes
airlineDataset_MixEncoding = pd.get_dummies(airlineDataset_MixEncoding, columns=['Gender', 'Customer Type', 'Type of Travel'])

Label Encoder nad celom skupu

In [ ]:
categorical_indexes = [0, 1, 3, 4] + list(range(6, 20))
airlineDataset_LabelEncoding.iloc[:,categorical_indexes] = airlineDataset_LabelEncoding.iloc[:,categorical_indexes].astype('object')
kategoricki_atributi_airline = airlineDataset_LabelEncoding.select_dtypes(include='object').columns.tolist()
kategoricki_atributi_airline
#airlineDataset.dtypes

In [ ]:
# Label Enkoder
for atribut in kategoricki_atributi_airline:
    label_encoder = LabelEncoder()
    label_encoder.fit(airlineDataset_LabelEncoding[atribut])
    airlineDataset_LabelEncoding[atribut] = label_encoder.transform(airlineDataset_LabelEncoding[atribut])
    airlineDataset_LabelEncoding[atribut] = airlineDataset_LabelEncoding[atribut].astype('int')
airlineDataset_LabelEncoding.head()

One Hot Encoder na celom skupu

In [ ]:
# One Hot Encoder
airlineDataset_OneHotEncoding = pd.get_dummies(data=airlineDataset_OneHotEncoding,columns=kategoricki_atributi_airline)
airlineDataset_OneHotEncoding.head()

<div style="border-radius:10px;
            background-color:#D6EAF8;
            letter-spacing:0.5px; text-align: center;">
            
<h3 style="padding: 5px 0px; color:#2980B9; font-weight: bold; font-family: Arial, sans-serif;">
Primena algoritma</h3>
</div>

In [ ]:
#Pravimo dataframe koji ce cuvati rezultate algoritma za kasnije poredjenje
testingDataframe = pd.DataFrame(columns=["Dataset","Enkodiranje", "Algoritam","F1", "Područje ispod ROC krive", "Accuracy"])

In [ ]:
def svm_klasifikacija(df,output,Dataset,Enkodiranje,testingDataframe):
    X = df.drop(output, axis=1)
    y = df[output]

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Create an SVM classifier
    svm = SVC(kernel='linear')

    # Train the SVM classifier on the training data
    svm.fit(X_train, y_train)

    # Make predictions on the testing data
    y_pred = svm.predict(X_test)

    # Calculate the accuracy of the classifier
    accuracy = accuracy_score(y_test, y_pred)*100
    roc = roc_auc_score(y_test,y_pred)
    f1 = f1_score(y_test,y_pred)

    testingDataframe.loc[-1] = [Dataset, Enkodiranje, "SVM", f1, roc, '{:.2f}%'.format(accuracy)]
    testingDataframe.index = testingDataframe.index + 1
    testingDataframe = testingDataframe.sort_index()

In [ ]:
def naive_bayes(df,output,Dataset,Enkodiranje,testingDataframe):
    X = df.drop(output, axis=1)
    y = df[output]

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Create a Naive Bayes classifier
    nb = GaussianNB()

    # Train the Naive Bayes classifier on the training data
    nb.fit(X_train, y_train)

    # Make predictions on the testing data
    y_pred = nb.predict(X_test)

    # Calculate the accuracy of the classifier
    accuracy = accuracy_score(y_test, y_pred)*100
    roc = roc_auc_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    testingDataframe.loc[-1] = [Dataset, Enkodiranje, "Naive Bayes", f1, roc, '{:.2f}%'.format(accuracy)]
    testingDataframe.index = testingDataframe.index + 1
    testingDataframe = testingDataframe.sort_index()

In [ ]:
def random_forest(df, output,Dataset, Enkodiranje,testingDataframe):
    X = df.drop(output, axis=1)
    y = df[output]

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Create a Random Forest classifier
    rf = RandomForestClassifier(n_estimators=100, random_state=42)

    # Train the Random Forest classifier on the training data
    rf.fit(X_train, y_train)

    # Make predictions on the testing data
    y_pred = rf.predict(X_test)

    # Calculate the accuracy of the classifier
    accuracy = accuracy_score(y_test, y_pred)*100
    roc = roc_auc_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    testingDataframe.loc[-1] = [Dataset, Enkodiranje, "Random Forest", f1, roc, '{:.2f}%'.format(accuracy)]
    testingDataframe.index = testingDataframe.index + 1
    testingDataframe = testingDataframe.sort_index()


In [ ]:
def neural_network(df, output,dataset,enkodiranje,testingDataframe):
    X = df.drop(output, axis=1)
    y = df[output]

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Define the model architecture
    model = Sequential()
    model.add(Dense(512, activation='relu', input_shape=(X_train.shape[1],)))
    model.add(Dropout(0.3))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    # Compile the model
    model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])

    # Set up early stopping to prevent overfitting
    es = EarlyStopping(monitor='val_accuracy', mode='max', patience=10, restore_best_weights=True)

    # Fit the model on the training data
    NN_Classifier = model.fit(X_train, y_train, batch_size=1000, epochs=1000, callbacks=[es], verbose=2, validation_data=(X_test, y_test))

    # Make predictions on the testing data
    y_pred = model.predict(X_test).round().astype(int)

    # Calculate the accuracy, AUC, and F1 score of the model
    accuracy = accuracy_score(y_test, y_pred)*100
    auc = roc_auc_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    # Add the model performance metrics to the testingDataframe
    testingDataframe.loc[-1] = [dataset, enkodiranje, "Neural network", f1, auc, '{:.2f}%'.format(accuracy)]
    testingDataframe.index = testingDataframe.index + 1
    testingDataframe = testingDataframe.sort_index()


  


<div style="border-radius:10px;
            background-color:#D6EAF8;
            letter-spacing:0.5px; text-align: center;">
            
<h3 style="padding: 5px 0px; color:#2980B9; font-weight: bold; font-family: Arial, sans-serif;">
Bank dataset</h3>
</div>

<div style="border-radius:10px;
            background-color:#D6EAF8;
            letter-spacing:0.5px; text-align: center;">
            
<h4 style="padding: 5px 0px; color:#2980B9; font-weight: bold; font-family: Arial, sans-serif;">
SVM</h4>
</div>

In [ ]:
svm_klasifikacija(bankDataset_MixEncoding,'y',"Bank","MIX",testingDataframe)
svm_klasifikacija(bankDataset_LabelEncoding,'y',"Bank","LabelEncoding",testingDataframe)
svm_klasifikacija(bankDataset_OneHotEncoding,'y',"Bank","OneHotEncoding",testingDataframe)

In [ ]:
testingDataframe

In [ ]:
#SVM on Mixed Encodings Bank dataset
# Extract the features and target variable
X_Bank_Mix = bankDataset_MixEncoding.drop('y', axis=1)
y_Bank_Mix = bankDataset_MixEncoding['y']

# Split the data into training and testing sets
X_train_BM, X_test_BM, y_train_BM, y_test_BM = train_test_split(X_Bank_Mix, y_Bank_Mix, test_size=0.2, random_state=42)

# Create an SVM classifier
svm = SVC(kernel='linear')

# Train the SVM classifier on the training data
svm.fit(X_train_BM, y_train_BM)

# Make predictions on the testing data
y_pred_BM = svm.predict(X_test_BM)

# Calculate the accuracy of the classifier
accuracy = accuracy_score(y_test_BM, y_pred_BM)*100
roc = roc_auc_score(y_test_BM,y_pred_BM)
f1 = f1_score(y_test_BM,y_pred_BM)


testingDataframe.loc[-1] = ["Bank dataset","MIX","SVM",f1,  roc, '{:.2f}%'.format(accuracy)]
testingDataframe.index = testingDataframe.index + 1
testing = testingDataframe.sort_index()
testing

In [ ]:
#SVM on All Label Encoded Bank dataset
# Extract the features and target variable
X_Bank_LE = bankDataset_LabelEncoding.drop('y', axis=1)
y_Bank_LE = bankDataset_LabelEncoding['y']

# Split the data into training and testing sets
X_train_BLE, X_test_BLE, y_train_BLE, y_test_BLE = train_test_split(X_Bank_LE, y_Bank_LE, test_size=0.2, random_state=42)

# Create an SVM classifier
svm = SVC(kernel='linear')

# Train the SVM classifier on the training data
svm.fit(X_train_BLE, y_train_BLE)

# Make predictions on the testing data
y_pred_BLE = svm.predict(X_test_BLE)

# Calculate the accuracy of the classifier
accuracy = accuracy_score(y_test_BLE, y_pred_BLE)*100
roc = roc_auc_score(y_test_BLE,y_pred_BLE)
f1 = f1_score(y_test_BLE,y_pred_BLE)


testingDataframe.loc[-1] = ["Bank dataset","Label Encoder","SVM",f1,  roc,  '{:.2f}%'.format(accuracy)]
testingDataframe.index = testingDataframe.index + 1
testing = testingDataframe.sort_index()
testing

In [ ]:
#SVM on All Hot Encoded Bank dataset
# Extract the features and target variable
X_Bank_OH = bankDataset_OneHotEncoding.drop(['y'], axis=1)
y_Bank_OH = bankDataset_OneHotEncoding['y']

# Split the data into training and testing sets
X_train_BOH, X_test_BOH, y_train_BOH, y_test_BOH = train_test_split(X_Bank_OH, y_Bank_OH, test_size=0.2, random_state=42)

# Create an SVM classifier
svm = SVC(kernel='linear')

# Train the SVM classifier on the training data
svm.fit(X_train_BOH, y_train_BOH)

# Make predictions on the testing data
y_pred_BOH = svm.predict(X_test_BOH)

# Calculate the accuracy of the classifier
accuracy = accuracy_score(y_test_BOH, y_pred_BOH)*100
roc = roc_auc_score(y_test_BOH,y_pred_BOH)
f1 = f1_score(y_test_BOH,y_pred_BOH)

testingDataframe.loc[-1] = ["Bank dataset","One Hot","SVM",f1,  roc,  '{:.2f}%'.format(accuracy)]
testingDataframe.index = testingDataframe.index + 1
testing = testingDataframe.sort_index()
testing

<div style="border-radius:10px;
            background-color:#D6EAF8;
            letter-spacing:0.5px; text-align: center;">
            
<h4 style="padding: 5px 0px; color:#2980B9; font-weight: bold; font-family: Arial, sans-serif;">
Naive Bayes</h4>
</div>

In [ ]:
naive_bayes(bankDataset_MixEncoding,'y',"Bank","MIX",testingDataframe)
naive_bayes(bankDataset_LabelEncoding,'y',"Bank","LabelEncoding",testingDataframe)
naive_bayes(bankDataset_OneHotEncoding,'y',"Bank","OneHotEncoding",testingDataframe)

In [ ]:
testingDataframe

In [ ]:
#NaiveBayes on Mixed Encodings Bank dataset
# Extract the features and target variable
X_Bank_Mix_NB = bankDataset_MixEncoding.drop('y', axis=1)
y_Bank_Mix_NB = bankDataset_MixEncoding['y']

# Split the data into training and testing sets
X_train_BM_NB, X_test_BM_NB, y_train_BM_NB, y_test_BM_NB = train_test_split(X_Bank_Mix_NB, y_Bank_Mix_NB, test_size=0.2, random_state=42)

# Create a Naive Bayes classifier
nb = GaussianNB()

# Train the Naive Bayes classifier on the training data
nb.fit(X_train_BM_NB, y_train_BM_NB)

# Make predictions on the testing data
y_pred_BM_NB = nb.predict(X_test_BM_NB)

# Calculate the accuracy of the classifier
accuracy = accuracy_score(y_test_BM_NB, y_pred_BM_NB)*100
roc = roc_auc_score(y_test_BM_NB, y_pred_BM_NB)
f1 = f1_score(y_test_BM_NB, y_pred_BM_NB)

testingDataframe.loc[-1] = ["Bank dataset","MIX","Naive Bayes",f1,  roc,  '{:.2f}%'.format(accuracy)]
testingDataframe.index = testingDataframe.index + 1
testing = testingDataframe.sort_index()
testing

In [ ]:
# NaiveBayes on All Label Encoded Bank dataset
#  Extract the features and target variable
X_Bank_LE_NB = bankDataset_LabelEncoding.drop('y', axis=1)
y_Bank_LE_NB = bankDataset_LabelEncoding['y']

# Split the data into training and testing sets
X_train_BLE_NB, X_test_BLE_NB, y_train_BLE_NB, y_test_BLE_NB = train_test_split(X_Bank_LE_NB, y_Bank_LE_NB, test_size=0.2, random_state=42)

# Create a Naive Bayes classifier
nb = GaussianNB()

# Train the Naive Bayes classifier on the training data
nb.fit(X_train_BLE_NB, y_train_BLE_NB)

# Make predictions on the testing data
y_pred_BLE_NB = nb.predict(X_test_BLE_NB)

# Calculate the accuracy of the classifier
accuracy = accuracy_score(y_test_BLE_NB, y_pred_BLE_NB)*100
roc = roc_auc_score(y_test_BLE_NB, y_pred_BLE_NB)
f1 = f1_score(y_test_BLE_NB, y_pred_BLE_NB)

testingDataframe.loc[-1] = ["Bank dataset","Label Encoder","Naive Bayes",f1,  roc,  '{:.2f}%'.format(accuracy)]
testingDataframe.index = testingDataframe.index + 1
testing = testingDataframe.sort_index()
testing

In [ ]:
# NaiveBayes on All One Hot Encodings Bank dataset
# Extract the features and target variable
X_Bank_OH_NB = bankDataset_OneHotEncoding.drop(['y'], axis=1)
y_Bank_OH_NB = bankDataset_OneHotEncoding['y']

# Split the data into training and testing sets
X_train_BOH_NB, X_test_BOH_NB, y_train_BOH_NB, y_test_BOH_NB = train_test_split(X_Bank_OH_NB, y_Bank_OH_NB, test_size=0.2, random_state=42)

# Create a Naive Bayes classifier
nb = GaussianNB()

# Train the Naive Bayes classifier on the training data
nb.fit(X_train_BOH_NB, y_train_BOH_NB)

# Make predictions on the testing data
y_pred_BOH_NB = nb.predict(X_test_BOH_NB)

# Calculate the accuracy of the classifier
accuracy = accuracy_score(y_test_BOH_NB, y_pred_BOH_NB)*100
roc = roc_auc_score(y_test_BOH_NB, y_pred_BOH_NB)
f1 = f1_score(y_test_BOH_NB, y_pred_BOH_NB)

testingDataframe.loc[-1] = ["Bank dataset","One Hot","Naive Bayes",f1, roc, '{:.2f}%'.format(accuracy)]
testingDataframe.index = testingDataframe.index + 1
testing = testingDataframe.sort_index()
testing

<div style="border-radius:10px;
            background-color:#D6EAF8;
            letter-spacing:0.5px; text-align: center;">
            
<h4 style="padding: 5px 0px; color:#2980B9; font-weight: bold; font-family: Arial, sans-serif;">
Random Forest</h4>
</div>

In [ ]:
random_forest(bankDataset_MixEncoding,'y',"Bank","MIX",testingDataframe)
random_forest(bankDataset_LabelEncoding,'y',"Bank","LabelEncoding",testingDataframe)
random_forest(bankDataset_OneHotEncoding,'y',"Bank","OneHotEncoding",testingDataframe)

In [ ]:
testingDataframe

In [ ]:
# Random Forest on Mixed Encoded Bank dataset
np.random.seed(42)

# Extract the features and target variable
X_Bank_Mix_RF = bankDataset_MixEncoding.drop(['y'], axis=1)
y_Bank_Mix_RF = bankDataset_MixEncoding['y']

# Split the data into training and testing sets
X_train_BM_RF, X_test_BM_RF, y_train_BM_RF, y_test_BM_RF = train_test_split(X_Bank_Mix_RF, y_Bank_Mix_RF, test_size=0.2, random_state=42)

randomForest_Bank_Mix = RandomForestClassifier()
randomForest_Bank_Mix.fit(X_train_BM_RF, y_train_BM_RF)

pred = randomForest_Bank_Mix.predict(X_test_BM_RF)
f1 = f1_score(y_test_BM_RF, randomForest_Bank_Mix.predict(X_test_BM_RF))
auc = roc_auc_score(y_test_BM_RF, randomForest_Bank_Mix.predict(X_test_BM_RF))
acc = accuracy_score(y_test_BM_RF, randomForest_Bank_Mix.predict(X_test_BM_RF))*100

testingDataframe.loc[-1] = ["Bank dataset","MIX","Random forest",f1,  auc,  '{:.2f}%'.format(acc)]
testingDataframe.index = testingDataframe.index + 1
testing = testingDataframe.sort_index()
testing

In [ ]:
# Random Forest on All Label Encoded Bank dataset
np.random.seed(42)

# Extract the features and target variable
X_Bank_LE_RF = bankDataset_LabelEncoding.drop(['y'], axis=1)
y_Bank_LE_RF = bankDataset_LabelEncoding['y']

# Split the data into training and testing sets
X_train_LE_RF, X_test_LE_RF, y_train_LE_RF, y_test_LE_RF = train_test_split(X_Bank_LE_RF, y_Bank_LE_RF, test_size=0.2, random_state=42)

randomForest_Bank_LE = RandomForestClassifier()
randomForest_Bank_LE.fit(X_train_LE_RF, y_train_LE_RF)

pred = randomForest_Bank_LE.predict(X_test_LE_RF)
f1 = f1_score(y_test_LE_RF, randomForest_Bank_LE.predict(X_test_LE_RF))
auc = roc_auc_score(y_test_LE_RF, randomForest_Bank_LE.predict(X_test_LE_RF))
acc = accuracy_score(y_test_LE_RF, randomForest_Bank_LE.predict(X_test_LE_RF))*100

testingDataframe.loc[-1] = ["Bank dataset","Label Encoder","Random forest",f1,  auc,  '{:.2f}%'.format(acc)]
testingDataframe.index = testingDataframe.index + 1
testing = testingDataframe.sort_index()
testing

In [ ]:
# Random Forest on All One Hot Encoded Bank dataset
np.random.seed(42)

# Extract the features and target variable
X_Bank_BOH_RF = bankDataset_OneHotEncoding.drop(['y'], axis=1)
y_Bank_BOH_RF = bankDataset_OneHotEncoding['y']

# Split the data into training and testing sets
X_train_BOH_RF, X_test_BOH_RF, y_train_BOH_RF, y_test_BOH_RF = train_test_split(X_Bank_BOH_RF, y_Bank_BOH_RF, test_size=0.2, random_state=42)

randomForest_Bank_LE = RandomForestClassifier()
randomForest_Bank_LE.fit(X_train_BOH_RF, y_train_BOH_RF)

pred = randomForest_Bank_LE.predict(X_test_BOH_RF)
f1 = f1_score(y_test_BOH_RF, randomForest_Bank_LE.predict(X_test_BOH_RF))
auc = roc_auc_score(y_test_BOH_RF, randomForest_Bank_LE.predict(X_test_BOH_RF))
acc = accuracy_score(y_test_BOH_RF, randomForest_Bank_LE.predict(X_test_BOH_RF))*100

testingDataframe.loc[-1] = ["Bank dataset","One Hot","Random forest",f1,  auc,  '{:.2f}%'.format(acc)]
testingDataframe.index = testingDataframe.index + 1
testing = testingDataframe.sort_index()
testing

<div style="border-radius:10px;
            background-color:#D6EAF8;
            letter-spacing:0.5px; text-align: center;">
            
<h4 style="padding: 5px 0px; color:#2980B9; font-weight: bold; font-family: Arial, sans-serif;">
Artificial Neural Networks</h4>
</div>

In [ ]:
neural_network(bankDataset_MixEncoding,'y',"Bank","MIX",testingDataframe)
neural_network(bankDataset_LabelEncoding,'y',"Bank","LabelEncoding",testingDataframe)
neural_network(bankDataset_OneHotEncoding,'y',"Bank","OneHotEncoding",testingDataframe)


In [ ]:
testingDataframe



In [ ]:
# ANN - Mixed Encoding - Bank dataset
X_Bank_BM_ANN = bankDataset_MixEncoding.drop(['y'], axis=1)
y_Bank_BM_ANN = bankDataset_MixEncoding['y']

# Split the data into training and testing sets
X_train_BM_ANN, X_test_BM_ANN, y_train_BM_ANN, y_test_BM_ANN = train_test_split(X_Bank_BM_ANN, y_Bank_BM_ANN, test_size=0.2, random_state=42)

model_BM_ANN = Sequential()
model_BM_ANN.add(Dense(512, activation='relu', input_shape=(X_train_BM_ANN.shape[1],)))
model_BM_ANN.add(Dropout(0.3))
model_BM_ANN.add(Dense(256, activation='relu'))
model_BM_ANN.add(Dense(256, activation='relu'))
model_BM_ANN.add(Dropout(0.3))
model_BM_ANN.add(Dense(128, activation='relu'))
model_BM_ANN.add(Dense(1, activation='sigmoid')) 

model_BM_ANN.compile(loss='binary_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

es = EarlyStopping(monitor='val_accuracy', 
                   mode='max', 
                   patience=10,  #Stop the model training if the validation accuracy doesnt increase in 10 Epochs
                   restore_best_weights=True)

NN_Classifier_BM = model_BM_ANN.fit(X_train_BM_ANN,y_train_BM_ANN,
                          batch_size=1000,
                          epochs=1000,
                          callbacks =[es],
                          verbose=2,
                          validation_data=(X_test_BM_ANN, y_test_BM_ANN))
                          

y_pred=(model_BM_ANN.predict(X_test_BM_ANN) > 0.5).astype(int)
acc = accuracy_score(y_test_BM_ANN, y_pred)*100
#print('Accuracy Score: ', '{:.2f}%'.format(acc))

testingDataframe.loc[-1] = ["Bank dataset","MIX","ANN",0,  0, '{:.2f}%'.format(acc)]
testingDataframe.index = testingDataframe.index + 1
testing = testingDataframe.sort_index()
testing

In [ ]:
# ANN - Label Encoding -  Bank dataset
X_Bank_BLE_ANN = bankDataset_LabelEncoding.drop(['y'], axis=1)
y_Bank_BLE_ANN = bankDataset_LabelEncoding['y']

# Split the data into training and testing sets
X_train_BLE_ANN, X_test_BLE_ANN, y_train_BLE_ANN, y_test_BLE_ANN = train_test_split(X_Bank_BLE_ANN, y_Bank_BLE_ANN, test_size=0.2, random_state=42)

model_BLE_ANN = Sequential()
model_BLE_ANN.add(Dense(512, activation='relu', input_shape=(X_train_BLE_ANN.shape[1],)))
model_BLE_ANN.add(Dropout(0.3))
model_BLE_ANN.add(Dense(256, activation='relu'))
model_BLE_ANN.add(Dense(256, activation='relu'))
model_BLE_ANN.add(Dropout(0.3))
model_BLE_ANN.add(Dense(128, activation='relu'))
model_BLE_ANN.add(Dense(1, activation='sigmoid')) 

model_BLE_ANN.compile(loss='binary_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

es = EarlyStopping(monitor='val_accuracy', 
                   mode='max', 
                   patience=10,  #Stop the model training if the validation accuracy doesnt increase in 10 Epochs
                   restore_best_weights=True)

NN_Classifier_BLE = model_BLE_ANN.fit(X_train_BLE_ANN, y_train_BLE_ANN,
                          batch_size=1000,
                          epochs=1000,
                          callbacks =[es],
                          verbose=2,
                          validation_data=(X_test_BLE_ANN, y_test_BLE_ANN))


y_pred=(model_BLE_ANN.predict(X_test_BLE_ANN) > 0.5).astype(int)
acc = accuracy_score(y_test_BLE_ANN, y_pred)*100

testingDataframe.loc[-1] = ["Bank dataset","Label Encoder","ANN",0,  0,  '{:.2f}%'.format(acc)]
testingDataframe.index = testingDataframe.index + 1
testing = testingDataframe.sort_index()
testing

In [ ]:
# ANN - One Hot Encoding -  Bank dataset
X_Bank_BOH_ANN = bankDataset_OneHotEncoding.drop(['y'], axis=1)
y_Bank_BOH_ANN = bankDataset_OneHotEncoding['y']

# Split the data into training and testing sets
X_train_BOH_ANN, X_test_BOH_ANN, y_train_BOH_ANN, y_test_BOH_ANN = train_test_split(X_Bank_BOH_ANN, y_Bank_BOH_ANN, test_size=0.2, random_state=42)

model_BOH_ANN = Sequential()
model_BOH_ANN.add(Dense(512, activation='relu', input_shape=(X_train_BOH_ANN.shape[1],)))
model_BOH_ANN.add(Dropout(0.3))
model_BOH_ANN.add(Dense(256, activation='relu'))
model_BOH_ANN.add(Dense(256, activation='relu'))
model_BOH_ANN.add(Dropout(0.3))
model_BOH_ANN.add(Dense(128, activation='relu'))
model_BOH_ANN.add(Dense(1, activation='sigmoid')) 

model_BOH_ANN.compile(loss='binary_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

es = EarlyStopping(monitor='val_accuracy', 
                   mode='max', 
                   patience=10,  #Stop the model training if the validation accuracy doesnt increase in 10 Epochs
                   restore_best_weights=True)

NN_Classifier_BOH = model_BOH_ANN.fit(X_train_BOH_ANN, y_train_BOH_ANN,
                          batch_size=1000,
                          epochs=1000,
                          callbacks =[es],
                          verbose=2,
                          validation_data=(X_test_BOH_ANN, y_test_BOH_ANN))


y_pred=(model_BOH_ANN.predict(X_test_BOH_ANN) > 0.5).astype(int)
acc = accuracy_score(y_test_BOH_ANN, y_pred)*100

testingDataframe.loc[-1] = ["Bank dataset","One Hot","ANN",0,  0,  '{:.2f}%'.format(acc)]
testingDataframe.index = testingDataframe.index + 1
testing = testingDataframe.sort_index()
testing

<div style="border-radius:10px;
            background-color:#D6EAF8;
            letter-spacing:0.5px; text-align: center;">
            
<h3 style="padding: 5px 0px; color:#2980B9; font-weight: bold; font-family: Arial, sans-serif;">
Airline dataset</h3>
</div>

<div style="border-radius:10px;
            background-color:#D6EAF8;
            letter-spacing:0.5px; text-align: center;">
            
<h4 style="padding: 5px 0px; color:#2980B9; font-weight: bold; font-family: Arial, sans-serif;">
Naive bayes</h4>
</div>

In [ ]:
naive_bayes(airlineDataset_MixEncoding,'satisfaction',"Airline","MIX",testingDataframe)
naive_bayes(airlineDataset_LabelEncoding,'satisfaction',"Airline","LabelEncoding",testingDataframe)
naive_bayes(airlineDataset_OneHotEncoding,'satisfaction',"Airline","OneHotEncoding",testingDataframe)

In [ ]:
testingDataframe

In [ ]:
# NaiveBayes - Mixed Encoding - Airline dataset

X_Airline_Mix_NB = airlineDataset_MixEncoding.drop('satisfaction', axis=1)
y_Airline_Mix_NB = airlineDataset_MixEncoding['satisfaction']

# Split the data into training and testing sets
X_train_AM_NB, X_test_AM_NB, y_train_AM_NB, y_test_AM_NB = train_test_split(X_Airline_Mix_NB, y_Airline_Mix_NB, test_size=0.2, random_state=42)

# Create a Naive Bayes classifier
nb = GaussianNB()

# Train the Naive Bayes classifier on the training data
nb.fit(X_train_AM_NB, y_train_AM_NB)

# Make predictions on the testing data
y_pred_AM_NB = nb.predict(X_test_AM_NB)

# Calculate the accuracy of the classifier
accuracy = accuracy_score(y_test_AM_NB, y_pred_AM_NB)*100
roc = roc_auc_score(y_test_AM_NB, y_pred_AM_NB)
f1 = f1_score(y_test_AM_NB, y_pred_AM_NB)

testingDataframe.loc[-1] = ["Airline dataset","MIX","Naive Bayes",f1,  roc,  '{:.2f}%'.format(accuracy)]
testingDataframe.index = testingDataframe.index + 1
testing = testingDataframe.sort_index()
testing

In [ ]:
# NaiveBayes - Label Encoding - Airline dataset

# Extract the features and target variable
X_Airline_LE_NB = airlineDataset_LabelEncoding.drop('satisfaction', axis=1)
y_Airline_LE_NB = airlineDataset_LabelEncoding['satisfaction']

# Split the data into training and testing sets
X_train_ALE_NB, X_test_ALE_NB, y_train_ALE_NB, y_test_ALE_NB = train_test_split(X_Airline_LE_NB, y_Airline_LE_NB, test_size=0.2, random_state=42)

# Create a Naive Bayes classifier
nb = GaussianNB()

# Train the Naive Bayes classifier on the training data
nb.fit(X_train_ALE_NB, y_train_ALE_NB)

# Make predictions on the testing data
y_pred_ALE_NB = nb.predict(X_test_ALE_NB)

# Calculate the accuracy of the classifier
accuracy = accuracy_score(y_test_ALE_NB, y_pred_ALE_NB)*100
roc = roc_auc_score(y_test_ALE_NB, y_pred_ALE_NB)
f1 = f1_score(y_test_ALE_NB, y_pred_ALE_NB)

testingDataframe.loc[-1] = ["Airline dataset","Label Encoder","Naive Bayes",f1,  roc,  '{:.2f}%'.format(accuracy)]
testingDataframe.index = testingDataframe.index + 1
testing = testingDataframe.sort_index()
testing

In [ ]:
# NaiveBayes - One Hot Encoding - Airline dataset

# Extract the features and target variable
X_Airline_OH_NB = airlineDataset_OneHotEncoding.drop(['satisfaction'], axis=1)
y_Airline_OH_NB = airlineDataset_OneHotEncoding['satisfaction']

# Split the data into training and testing sets
X_train_AOH_NB, X_test_AOH_NB, y_train_AOH_NB, y_test_AOH_NB = train_test_split(X_Airline_OH_NB, y_Airline_OH_NB, test_size=0.2, random_state=42)

# Create a Naive Bayes classifier
nb = GaussianNB()

# Train the Naive Bayes classifier on the training data
nb.fit(X_train_AOH_NB, y_train_AOH_NB)

# Make predictions on the testing data
y_pred_AOH_NB = nb.predict(X_test_AOH_NB)

# Calculate the accuracy of the classifier
accuracy = accuracy_score(y_test_AOH_NB, y_pred_AOH_NB)*100
roc = roc_auc_score(y_test_AOH_NB, y_pred_AOH_NB)
f1 = f1_score(y_test_AOH_NB, y_pred_AOH_NB)

testingDataframe.loc[-1] = ["Airline dataset","One Hot","Naive Bayes",f1, roc, '{:.2f}%'.format(accuracy)]
testingDataframe.index = testingDataframe.index + 1
testing = testingDataframe.sort_index()
testing

<div style="border-radius:10px;
            background-color:#D6EAF8;
            letter-spacing:0.5px; text-align: center;">
            
<h4 style="padding: 5px 0px; color:#2980B9; font-weight: bold; font-family: Arial, sans-serif;">
Random Forest</h4>
</div>

In [ ]:
random_forest(airlineDataset_MixEncoding,'satisfaction',"Airline","MIX",testingDataframe)
random_forest(airlineDataset_LabelEncoding,'satisfaction',"Airline","LabelEncoding",testingDataframe)
random_forest(airlineDataset_OneHotEncoding,'satisfaction',"Airline","OneHotEncoding",testingDataframe)

In [ ]:
# RandomForest - Mixed Encoding - Airline dataset

np.random.seed(42)

# Extract the features and target variable
X_Airline_Mix_RF = airlineDataset_MixEncoding.drop(['satisfaction'], axis=1)
y_Airline_Mix_RF = airlineDataset_MixEncoding['satisfaction']

# Split the data into training and testing sets
X_train_AM_RF, X_test_AM_RF, y_train_AM_RF, y_test_AM_RF = train_test_split(X_Airline_Mix_RF, y_Airline_Mix_RF, test_size=0.2, random_state=42)

randomForest_Airline_Mix = RandomForestClassifier()
randomForest_Airline_Mix.fit(X_train_AM_RF, y_train_AM_RF)

pred = randomForest_Airline_Mix.predict(X_test_AM_RF)
f1 = f1_score(y_test_AM_RF, randomForest_Airline_Mix.predict(X_test_AM_RF))
auc = roc_auc_score(y_test_AM_RF, randomForest_Airline_Mix.predict(X_test_AM_RF))
acc = accuracy_score(y_test_AM_RF, randomForest_Airline_Mix.predict(X_test_AM_RF))*100

testingDataframe.loc[-1] = ["Airline dataset","MIX","Random forest",f1,  auc,  '{:.2f}%'.format(acc)]
testingDataframe.index = testingDataframe.index + 1
testing = testingDataframe.sort_index()
testing

In [ ]:
# RandomForest - Label Encoding - Airline dataset


# Extract the features and target variable
X_Airline_LE_RF = airlineDataset_LabelEncoding.drop(['satisfaction'], axis=1)
y_Airline_LE_RF = airlineDataset_LabelEncoding['satisfaction']

# Split the data into training and testing sets
X_train_ALE_RF, X_test_ALE_RF, y_train_ALE_RF, y_test_ALE_RF = train_test_split(X_Airline_LE_RF, y_Airline_LE_RF, test_size=0.2, random_state=42)

randomForest_Airline_LE = RandomForestClassifier()
randomForest_Airline_LE.fit(X_train_ALE_RF, y_train_ALE_RF)

pred = randomForest_Airline_LE.predict(X_test_ALE_RF)
f1 = f1_score(y_test_ALE_RF, randomForest_Airline_LE.predict(X_test_ALE_RF))
auc = roc_auc_score(y_test_ALE_RF, randomForest_Airline_LE.predict(X_test_ALE_RF))
acc = accuracy_score(y_test_ALE_RF, randomForest_Airline_LE.predict(X_test_ALE_RF))*100

testingDataframe.loc[-1] = ["Airline dataset","Label Encoder","Random forest",f1,  auc,  '{:.2f}%'.format(acc)]
testingDataframe.index = testingDataframe.index + 1
testing = testingDataframe.sort_index()
testing

In [ ]:
# RandomForest - OneHot Encoding - Airline dataset

np.random.seed(42)

# Extract the features and target variable
X_Airline_BOH_RF = airlineDataset_OneHotEncoding.drop(['satisfaction'], axis=1)
y_Airline_BOH_RF = airlineDataset_OneHotEncoding['satisfaction']

# Split the data into training and testing sets
X_train_AOH_RF, X_test_AOH_RF, y_train_AOH_RF, y_test_AOH_RF = train_test_split(X_Airline_BOH_RF, y_Airline_BOH_RF, test_size=0.2, random_state=42)

randomForest_Airline_OH = RandomForestClassifier()
randomForest_Airline_OH.fit(X_train_AOH_RF, y_train_AOH_RF)

pred = randomForest_Airline_OH.predict(X_test_AOH_RF)
f1 = f1_score(y_test_AOH_RF, randomForest_Airline_OH.predict(X_test_AOH_RF))
auc = roc_auc_score(y_test_AOH_RF, randomForest_Airline_OH.predict(X_test_AOH_RF))
acc = accuracy_score(y_test_AOH_RF, randomForest_Airline_OH.predict(X_test_AOH_RF))*100

testingDataframe.loc[-1] = ["Airline dataset","One Hot","Random forest",f1,  auc,  '{:.2f}%'.format(acc)]
testingDataframe.index = testingDataframe.index + 1
testing = testingDataframe.sort_index()
testing

<div style="border-radius:10px;
            background-color:#D6EAF8;
            letter-spacing:0.5px; text-align: center;">
            
<h4 style="padding: 5px 0px; color:#2980B9; font-weight: bold; font-family: Arial, sans-serif;">
Artificial Neural Network</h4>
</div>

In [ ]:
neural_network(airlineDataset_MixEncoding,'satisfaction',"Airline","MIX",testingDataframe)
neural_network(airlineDataset_LabelEncoding,'satisfaction',"Airline","LabelEncoding",testingDataframe)
neural_network(airlineDataset_OneHotEncoding,'satisfaction',"Airline","OneHotEncoding",testingDataframe)

In [ ]:
testingDataframe

In [ ]:
# ANN - Mixed Encoding - Airline dataset

def naive_bayes(df,output):
    X = df.drop(output, axis=1)
    y = df[output]

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Create a Naive Bayes classifier
    nb = GaussianNB()

    # Train the Naive Bayes classifier on the training data
    nb.fit(X_train, y_train)

    # Make predictions on the testing data
    y_pred = nb.predict(X_test)

    # Calculate the accuracy of the classifier
    accuracy = accuracy_score(y_test, y_pred)*100
    roc = roc_auc_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
#print('Accuracy Score: ', '{:.2f}%'.format(acc))

testingDataframe.loc[-1] = ["Airline dataset","MIX","ANN",0,  0, '{:.2f}%'.format(acc)]
testingDataframe.index = testingDataframe.index + 1
testing = testingDataframe.sort_index()
testing

In [ ]:
# ANN - Label Encoding - Airline dataset


X_Airline_BLE_ANN = airlineDataset_LabelEncoding.drop(['satisfaction'], axis=1)
y_Airline_BLE_ANN = airlineDataset_LabelEncoding['satisfaction']

# Split the data into training and testing sets
X_train_ALE_ANN, X_test_ALE_ANN, y_train_ALE_ANN, y_test_ALE_ANN = train_test_split(X_Airline_BLE_ANN, y_Airline_BLE_ANN, test_size=0.2, random_state=42)

model_ALE_ANN = Sequential()
model_ALE_ANN.add(Dense(512, activation='relu', input_shape=(X_train_ALE_ANN.shape[1],)))
model_ALE_ANN.add(Dropout(0.3))
model_ALE_ANN.add(Dense(256, activation='relu'))
model_ALE_ANN.add(Dense(256, activation='relu'))
model_ALE_ANN.add(Dropout(0.3))
model_ALE_ANN.add(Dense(128, activation='relu'))
model_ALE_ANN.add(Dense(1, activation='sigmoid')) 

model_ALE_ANN.compile(loss='binary_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

es = EarlyStopping(monitor='val_accuracy', 
                   mode='max', 
                   patience=10,  #Stop the model training if the validation accuracy doesnt increase in 10 Epochs
                   restore_best_weights=True)

NN_Classifier_ALE = model_ALE_ANN.fit(X_train_ALE_ANN, y_train_ALE_ANN,
                          batch_size=1000,
                          epochs=1000,
                          callbacks =[es],
                          verbose=2,
                          validation_data=(X_test_ALE_ANN, y_test_ALE_ANN))


y_pred=(model_ALE_ANN.predict(X_test_ALE_ANN) > 0.5).astype(int)
acc = accuracy_score(y_test_ALE_ANN, y_pred)*100

testingDataframe.loc[-1] = ["Airline dataset","Label Encoder","ANN",0,  0,  '{:.2f}%'.format(acc)]
testingDataframe.index = testingDataframe.index + 1
testing = testingDataframe.sort_index()
testing

In [ ]:
# ANN - OneHot Encoding - Airline dataset

X_Airline_AOH_ANN = airlineDataset_OneHotEncoding.drop(['satisfaction'], axis=1)
y_Airline_AOH_ANN = airlineDataset_OneHotEncoding['satisfaction']

# Split the data into training and testing sets
X_train_AOH_ANN, X_test_AOH_ANN, y_train_AOH_ANN, y_test_AOH_ANN = train_test_split(X_Airline_AOH_ANN, y_Airline_AOH_ANN, test_size=0.2, random_state=42)

model_AOH_ANN = Sequential()
model_AOH_ANN.add(Dense(512, activation='relu', input_shape=(X_train_AOH_ANN.shape[1],)))
model_AOH_ANN.add(Dropout(0.3))
model_AOH_ANN.add(Dense(256, activation='relu'))
model_AOH_ANN.add(Dense(256, activation='relu'))
model_AOH_ANN.add(Dropout(0.3))
model_AOH_ANN.add(Dense(128, activation='relu'))
model_AOH_ANN.add(Dense(1, activation='sigmoid')) 

model_AOH_ANN.compile(loss='binary_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

es = EarlyStopping(monitor='val_accuracy', 
                   mode='max', 
                   patience=10,  #Stop the model training if the validation accuracy doesnt increase in 10 Epochs
                   restore_best_weights=True)

NN_Classifier_AOH = model_AOH_ANN.fit(X_train_AOH_ANN, y_train_AOH_ANN,
                          batch_size=1000,
                          epochs=1000,
                          callbacks =[es],
                          verbose=2,
                          validation_data=(X_test_AOH_ANN, y_test_AOH_ANN))


y_pred=(model_AOH_ANN.predict(X_test_AOH_ANN) > 0.5).astype(int)
acc = accuracy_score(y_test_AOH_ANN, y_pred)*100

testingDataframe.loc[-1] = ["Airline dataset","One Hot","ANN",0,  0,  '{:.2f}%'.format(acc)]
testingDataframe.index = testingDataframe.index + 1
testing = testingDataframe.sort_index()
testing

<div style="border-radius:10px;
            background-color:#D6EAF8;
            letter-spacing:0.5px; text-align: center;">
            
<h3 style="padding: 5px 0px; color:#2980B9; font-weight: bold; font-family: Arial, sans-serif;">
Poredjenje rezultata</h3>
</div>

In [ ]:
# Sorting by Accuracy
testing = testingDataframe.sort_values(by=['Accuracy'],ascending=False)
testing

In [ ]:
# Sorting by F1
testing = testingDataframe.sort_values(by=['F1'],ascending=False)
testing

In [ ]:
# Sorting by ROC
testing = testingDataframe.sort_values(by=['Područje ispod ROC krive'],ascending=False)
testing

<div style="border-radius:10px;
            background-color:#D6EAF8;
            letter-spacing:0.5px; text-align: center;">
            
<h2 style="padding: 5px 0px; color:#2980B9; font-weight: bold; font-family: Arial, sans-serif;">
Skaliranje podataka</h2>
</div>

<div style="border-radius:10px;
            background-color:#D6EAF8;
            letter-spacing:0.5px; text-align: left;
            padding-left: 20px">
            
<p style="padding: 5px 0px; color:#2980B9; font-family: Arial, sans-serif;">
U teorijskom delu rada, bilo je reči o razlikama između standardizacije i normalizacije podataka. Predstavljene su dobre i loše strane kao i tehnike koje se koriste. Napravljena je podela i rečeno je da su tehnike koje se korista za normalizaciju: <br>
    1. MinMax Scaler<br>
    2. MaxAbs Scaler<br>
    3. Robust Scaler<br>
A tehnika koja se koristi za standardizaciju:<br>
    <t>1. Standard Scaler<br></p>
</div>


<div style="border-radius:10px;
            background-color:#D6EAF8;
            letter-spacing:0.5px; text-align: center;">
            
<h3 style="padding: 5px 0px; color:#2980B9; font-weight: bold; font-family: Arial, sans-serif;">
Tehnike normalizacije podataka</h3>
</div>

In [ ]:
# Normalizovanje podataka MinMaxScaler-om
from sklearn.preprocessing import MinMaxScaler

def normalize_data(df,output):
    scaler = MinMaxScaler()
    normalized_df = (
        df.drop(columns=[output])
        .pipe(lambda df: pd.DataFrame(scaler.fit_transform(df), columns=df.columns))
        .assign(y=df[output])
    )
    return normalized_df


In [ ]:
# Normalizovanje podataka MaxAbsScaler-om
from sklearn.preprocessing import MaxAbsScaler

def normalize_data_maxabs(df,output):
    scaler = MaxAbsScaler()
    normalized_df = (
        df.drop(columns=[output])
        .pipe(lambda df: pd.DataFrame(scaler.fit_transform(df), columns=df.columns))
        .assign(y=df[output])
    )
    return normalized_df


In [ ]:
# Normalizovanje podataka RobustScaler-om
from sklearn.preprocessing import RobustScaler

def normalize_data_robust(df,output):
    scaler = RobustScaler()
    normalized_df = (
        df.drop(columns=[output])
        .pipe(lambda df: pd.DataFrame(scaler.fit_transform(df), columns=df.columns))
        .assign(y=df[output])
    )
    return normalized_df


<div style="border-radius:10px;
            background-color:#D6EAF8;
            letter-spacing:0.5px; text-align: center;">
            
<h3 style="padding: 5px 0px; color:#2980B9; font-weight: bold; font-family: Arial, sans-serif;">
Tehnike standardizacije podataka</h3>
</div>

In [ ]:
# Standardizovanje podataka StandardScaler-om
from sklearn.preprocessing import StandardScaler

def standardize_data(df,output):
    scaler = StandardScaler()
    normalized_df = (
        df.drop(columns=[output])
        .pipe(lambda df: pd.DataFrame(scaler.fit_transform(df), columns=df.columns))
        .assign(y=df[output])
    )
    return normalized_df


<div style="border-radius:10px;
            background-color:#D6EAF8;
            letter-spacing:0.5px; text-align: center;">
            
<h3 style="padding: 5px 0px; color:#2980B9; font-weight: bold; font-family: Arial, sans-serif;">
Normalizovanje Bank dataseta</h3>
</div>

In [ ]:
# Primena MinMaxScalera na bank datasetu

# Kombinovano enkodiranje
normalized_bank_mix = normalize_data(bankDataset_MixEncoding,'y')
# LabelEncoder
normalized_bank_le = normalize_data(bankDataset_LabelEncoding,'y')
# OneHotEncoder
normalized_bank_oh = normalize_data(bankDataset_OneHotEncoding,'y')


<div style="border-radius:10px;
            background-color:#D6EAF8;
            letter-spacing:0.5px; text-align: center;">
            
<h3 style="padding: 5px 0px; color:#2980B9; font-weight: bold; font-family: Arial, sans-serif;">
Standardizovanje Bank dataseta</h3>
</div>

In [ ]:
# Primena StandardScalera na bank datasetu
standardised_bank_mix = standardize_data(bankDataset_MixEncoding,'y')
standardised_bank_le = standardize_data(bankDataset_LabelEncoding,'y')
standardised_bank_oh = standardize_data(bankDataset_OneHotEncoding,'y')

<div style="border-radius:10px;
            background-color:#D6EAF8;
            letter-spacing:0.5px; text-align: center;">
            
<h3 style="padding: 5px 0px; color:#2980B9; font-weight: bold; font-family: Arial, sans-serif;">
Standardizovanje Airline dataseta</h3>
</div>

In [ ]:
standardized_airline_mix = standardize_data(airlineDataset_MixEncoding,'satisfaction')
standardized_airline_le = standardize_data(airlineDataset_LabelEncoding,'satisfaction')
standardized_airline_oh = standardize_data(airlineDataset_OneHotEncoding,'satisfaction')

<div style="border-radius:10px;
            background-color:#D6EAF8;
            letter-spacing:0.5px; text-align: center;">
            
<h3 style="padding: 5px 0px; color:#2980B9; font-weight: bold; font-family: Arial, sans-serif;">
Normalizovanje Airline dataseta</h3>
</div>

In [ ]:
# Normalizing Mixed Encoded dataset
normalised_airline_mix = normalize_data(airlineDataset_MixEncoding,'satisfaction')
# Normalizing Label Encoded dataset
normalised_airline_le = normalize_data(airlineDataset_LabelEncoding,'satisfaction')
# Normalizing OneHot Encoded dataset
normalised_airline_oh = normalize_data(airlineDataset_OneHotEncoding,'satisfaction')

<div style="border-radius:10px;
            background-color:#D6EAF8;
            letter-spacing:0.5px; text-align: center;">
            
<h3 style="padding: 5px 0px; color:#2980B9; font-weight: bold; font-family: Arial, sans-serif;">
Testiranje SVM na Normalizovanom Bank datasetu</h3>
</div>

In [ ]:
# SVM - Mixed Encoding - Airline normalized
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score


# Extract the features and target variable
X = normalised_bank_mix.drop(['y'],axis=1)
y = normalised_bank_mix['y']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create an SVM classifier
svm = SVC(kernel='linear')

# Train the SVM classifier on the training data
svm.fit(X_train, y_train)

# Make predictions on the testing data
y_pred = svm.predict(X_test)

# Calculate the accuracy of the classifier
accuracy = accuracy_score(y_test, y_pred)*100
f1  = f1_score(y_test,y_pred)
roc = roc_auc_score(y_test,y_pred)

testingDataframe.loc[-1] = ["Bank Normalizovan","MIX","SVM",f1,  roc,  '{:.2f}%'.format(accuracy)]
testingDataframe.index = testingDataframe.index + 1
testing = testingDataframe.sort_index()
testing

In [ ]:
# SVM - Label Encoding - Airline normalized

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score


# Extract the features and target variable
X = normalised_bank_le.drop('y',axis=1)
y = normalised_bank_le['y']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create an SVM classifier
svm = SVC(kernel='linear')

# Train the SVM classifier on the training data
svm.fit(X_train, y_train)

# Make predictions on the testing data
y_pred = svm.predict(X_test)

# Calculate the accuracy of the classifier
accuracy = accuracy_score(y_test, y_pred)*100
f1  = f1_score(y_test,y_pred)
roc = roc_auc_score(y_test,y_pred)

testingDataframe.loc[-1] = ["Bank Normalizovan","Label Encoder","SVM",f1,  roc,  '{:.2f}%'.format(accuracy)]
testingDataframe.index = testingDataframe.index + 1
testing = testingDataframe.sort_index()
testing

In [ ]:
# SVM - OneHot Encoding - Airline normalized

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score


# Extract the features and target variable
X_OH = normalised_bank_oh.drop('y',axis=1)
y_OH = normalised_bank_oh['y']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_OH, y_OH, test_size=0.2, random_state=42)

# Create an SVM classifier
svm = SVC(kernel='linear')

# Train the SVM classifier on the training data
svm.fit(X_train, y_train)

# Make predictions on the testing data
y_pred = svm.predict(X_test)

# Calculate the accuracy of the classifier
accuracy = accuracy_score(y_test, y_pred)*100
f1  = f1_score(y_test,y_pred)
roc = roc_auc_score(y_test,y_pred)

testingDataframe.loc[-1] = ["Bank Normalizovan","One Hot","SVM",f1,  roc,  '{:.2f}%'.format(accuracy)]
testingDataframe.index = testingDataframe.index + 1
testing = testingDataframe.sort_index()
testing

<div style="border-radius:10px;
            background-color:#D6EAF8;
            letter-spacing:0.5px; text-align: center;">
            
<h3 style="padding: 5px 0px; color:#2980B9; font-weight: bold; font-family: Arial, sans-serif;">
Testiranje Naive Bayes na Normalizovanom Bank datasetu</h3>
</div>

In [ ]:
#NaiveBayes on Mixed Encodings Bank dataset
# Extract the features and target variable
X_Bank_Mix_NB_N = normalised_bank_mix.drop('y',axis=1)
y_Bank_Mix_NB_N = normalised_bank_mix['y']

# Split the data into training and testing sets
X_train_BM_NB, X_test_BM_NB, y_train_BM_NB, y_test_BM_NB = train_test_split(X_Bank_Mix_NB_N, y_Bank_Mix_NB_N, test_size=0.2, random_state=42)

# Create a Naive Bayes classifier
nb = GaussianNB()

# Train the Naive Bayes classifier on the training data
nb.fit(X_train_BM_NB, y_train_BM_NB)

# Make predictions on the testing data
y_pred_BM_NB = nb.predict(X_test_BM_NB)

# Calculate the accuracy of the classifier
accuracy = accuracy_score(y_test_BM_NB, y_pred_BM_NB)*100
roc = roc_auc_score(y_test_BM_NB, y_pred_BM_NB)
f1 = f1_score(y_test_BM_NB, y_pred_BM_NB)

testingDataframe.loc[-1] = ["Bank normalizovan","MIX","Naive Bayes",f1,  roc,  '{:.2f}%'.format(accuracy)]
testingDataframe.index = testingDataframe.index + 1
testing = testingDataframe.sort_index()
testing

In [ ]:
#NaiveBayes on Mixed Encodings Bank dataset
# Extract the features and target variable
X_Bank_Mix_NB_N = normalised_bank_le.drop('y',axis=1)
y_Bank_Mix_NB_N = normalised_bank_le['y']

# Split the data into training and testing sets
X_train_BM_NB, X_test_BM_NB, y_train_BM_NB, y_test_BM_NB = train_test_split(X_Bank_Mix_NB_N, y_Bank_Mix_NB_N, test_size=0.2, random_state=42)

# Create a Naive Bayes classifier
nb = GaussianNB()

# Train the Naive Bayes classifier on the training data
nb.fit(X_train_BM_NB, y_train_BM_NB)

# Make predictions on the testing data
y_pred_BM_NB = nb.predict(X_test_BM_NB)

# Calculate the accuracy of the classifier
accuracy = accuracy_score(y_test_BM_NB, y_pred_BM_NB)*100
roc = roc_auc_score(y_test_BM_NB, y_pred_BM_NB)
f1 = f1_score(y_test_BM_NB, y_pred_BM_NB)

testingDataframe.loc[-1] = ["Bank normalizovan","Label Encoder","Naive Bayes",f1,  roc,  '{:.2f}%'.format(accuracy)]
testingDataframe.index = testingDataframe.index + 1
testing = testingDataframe.sort_index()
testing

In [ ]:
#NaiveBayes on Mixed Encodings Bank dataset
# Extract the features and target variable
X_Bank_Mix_NB_O = normalised_bank_oh.drop('y',axis=1)
y_Bank_Mix_NB_O = normalised_bank_oh['y']

# Split the data into training and testing sets
X_train_BM_NB, X_test_BM_NB, y_train_BM_NB, y_test_BM_NB = train_test_split(X_Bank_Mix_NB_O, y_Bank_Mix_NB_O, test_size=0.2, random_state=42)

# Create a Naive Bayes classifier
nb = GaussianNB()

# Train the Naive Bayes classifier on the training data
nb.fit(X_train_BM_NB, y_train_BM_NB)

# Make predictions on the testing data
y_pred_BM_NB = nb.predict(X_test_BM_NB)

# Calculate the accuracy of the classifier
accuracy = accuracy_score(y_test_BM_NB, y_pred_BM_NB)*100
roc = roc_auc_score(y_test_BM_NB, y_pred_BM_NB)
f1 = f1_score(y_test_BM_NB, y_pred_BM_NB)

testingDataframe.loc[-1] = ["Bank normalizovan","One Hot","Naive Bayes",f1,  roc,  '{:.2f}%'.format(accuracy)]
testingDataframe.index = testingDataframe.index + 1
testing = testingDataframe.sort_index()
testing

<div style="border-radius:10px;
            background-color:#D6EAF8;
            letter-spacing:0.5px; text-align: center;">
            
<h3 style="padding: 5px 0px; color:#2980B9; font-weight: bold; font-family: Arial, sans-serif;">
Random Forest na normalizovanom Bank datasetu</h3>
</div>

In [ ]:
# RandomForest - Mixed Encoding - Airline normalized

np.random.seed(42)

# Extract the features and target variable
X_Airline_Mix_RF = normalised_bank_mix.drop('y',axis=1)
y_Airline_Mix_RF = normalised_bank_mix['y']

# Split the data into training and testing sets
X_train_AM_RF, X_test_AM_RF, y_train_AM_RF, y_test_AM_RF = train_test_split(X_Airline_Mix_RF, y_Airline_Mix_RF, test_size=0.2, random_state=42)

randomForest_Airline_Mix = RandomForestClassifier()
randomForest_Airline_Mix.fit(X_train_AM_RF, y_train_AM_RF)

pred = randomForest_Airline_Mix.predict(X_test_AM_RF)
f1 = f1_score(y_test_AM_RF, randomForest_Airline_Mix.predict(X_test_AM_RF))
auc = roc_auc_score(y_test_AM_RF, randomForest_Airline_Mix.predict(X_test_AM_RF))
acc = accuracy_score(y_test_AM_RF, randomForest_Airline_Mix.predict(X_test_AM_RF))*100

testingDataframe.loc[-1] = ["Bank Normalizovan","MIX","Random forest",f1,  auc,  '{:.2f}%'.format(acc)]
testingDataframe.index = testingDataframe.index + 1
testing = testingDataframe.sort_index()
testing

In [ ]:
# RandomForest - Label Encoding - Bank normalized

np.random.seed(42)

# Extract the features and target variable
X_Airline_Mix_RF = normalised_bank_le.drop('y',axis=1)
y_Airline_Mix_RF = normalised_bank_le['y']

# Split the data into training and testing sets
X_train_AM_RF, X_test_AM_RF, y_train_AM_RF, y_test_AM_RF = train_test_split(X_Airline_Mix_RF, y_Airline_Mix_RF, test_size=0.2, random_state=42)

randomForest_Airline_Mix = RandomForestClassifier()
randomForest_Airline_Mix.fit(X_train_AM_RF, y_train_AM_RF)

pred = randomForest_Airline_Mix.predict(X_test_AM_RF)
f1 = f1_score(y_test_AM_RF, randomForest_Airline_Mix.predict(X_test_AM_RF))
auc = roc_auc_score(y_test_AM_RF, randomForest_Airline_Mix.predict(X_test_AM_RF))
acc = accuracy_score(y_test_AM_RF, randomForest_Airline_Mix.predict(X_test_AM_RF))*100

testingDataframe.loc[-1] = ["Bank Normalizovan","Label Encoder","Random forest",f1,  auc,  '{:.2f}%'.format(acc)]
testingDataframe.index = testingDataframe.index + 1
testing = testingDataframe.sort_index()
testing

In [ ]:
# RandomForest - OneHot Encoding - Airline normalized

np.random.seed(42)

# Extract the features and target variable
X_Airline_Mix_RF = normalised_bank_oh.drop('y',axis=1)
y_Airline_Mix_RF = normalised_bank_oh['y']

# Split the data into training and testing sets
X_train_AM_RF, X_test_AM_RF, y_train_AM_RF, y_test_AM_RF = train_test_split(X_Airline_Mix_RF, y_Airline_Mix_RF, test_size=0.2, random_state=42)

randomForest_Airline_Mix = RandomForestClassifier()
randomForest_Airline_Mix.fit(X_train_AM_RF, y_train_AM_RF)

pred = randomForest_Airline_Mix.predict(X_test_AM_RF)
f1 = f1_score(y_test_AM_RF, randomForest_Airline_Mix.predict(X_test_AM_RF))
auc = roc_auc_score(y_test_AM_RF, randomForest_Airline_Mix.predict(X_test_AM_RF))
acc = accuracy_score(y_test_AM_RF, randomForest_Airline_Mix.predict(X_test_AM_RF))*100

testingDataframe.loc[-1] = ["Bank Normalizovan","One Hot","Random forest",f1,  auc,  '{:.2f}%'.format(acc)]
testingDataframe.index = testingDataframe.index + 1
testing = testingDataframe.sort_index()
testing

<div style="border-radius:10px;
            background-color:#D6EAF8;
            letter-spacing:0.5px; text-align: center;">
            
<h3 style="padding: 5px 0px; color:#2980B9; font-weight: bold; font-family: Arial, sans-serif;">
ANN nad Bank normalizovanim skupom podataka</h3>
</div>

In [ ]:
# ANN - Mixed Encoding - Airline normalized

X_Bank_AM_ANN_N = normalised_bank_mix.drop(['y'],axis=1)
y_Bank_AM_ANN_N = normalised_bank_mix['y']

# Split the data into training and testing sets
X_train_AM_ANN, X_test_AM_ANN, y_train_AM_ANN, y_test_AM_ANN = train_test_split(X_Bank_AM_ANN_N, y_Bank_AM_ANN_N, test_size=0.2, random_state=42)

model_AM_ANN = Sequential()
model_AM_ANN.add(Dense(512, activation='relu', input_shape=(X_train_AM_ANN.shape[1],)))
model_AM_ANN.add(Dropout(0.3))
model_AM_ANN.add(Dense(256, activation='relu'))
model_AM_ANN.add(Dense(256, activation='relu'))
model_AM_ANN.add(Dropout(0.3))
model_AM_ANN.add(Dense(128, activation='relu'))
model_AM_ANN.add(Dense(1, activation='sigmoid')) 

model_AM_ANN.compile(loss='binary_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

es = EarlyStopping(monitor='val_accuracy', 
                   mode='max', 
                   patience=10,  #Stop the model training if the validation accuracy doesnt increase in 10 Epochs
                   restore_best_weights=True)

NN_Classifier_AM_N = model_AM_ANN.fit(X_train_AM_ANN,y_train_AM_ANN,
                          batch_size=1000,
                          epochs=1000,
                          callbacks =[es],
                          verbose=2,
                          validation_data=(X_test_AM_ANN, y_test_AM_ANN))
                          

y_pred=(model_AM_ANN.predict(X_test_AM_ANN) > 0.5).astype(int)
acc = accuracy_score(y_test_AM_ANN, y_pred)*100
#print('Accuracy Score: ', '{:.2f}%'.format(acc))

testingDataframe.loc[-1] = ["Bank normalizovan","MIX","ANN",0,  0, '{:.2f}%'.format(acc)]
testingDataframe.index = testingDataframe.index + 1
testing = testingDataframe.sort_index()
testing

In [ ]:
# ANN - Label Encoding - Airline normalized
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
X_Bank_BM_ANN = normalised_bank_le.drop('y',axis=1)
y_Bank_BM_ANN = normalised_bank_le['y']

# Split the data into training and testing sets
X_train_AM_ANN, X_test_AM_ANN, y_train_AM_ANN, y_test_AM_ANN = train_test_split(X_Bank_BM_ANN, y_Bank_BM_ANN, test_size=0.2, random_state=42)

model_AM_ANN = Sequential()
model_AM_ANN.add(Dense(512, activation='relu', input_shape=(X_train_AM_ANN.shape[1],)))
model_AM_ANN.add(Dropout(0.3))
model_AM_ANN.add(BatchNormalization())
model_AM_ANN.add(Dense(256, activation='relu'))
model_AM_ANN.add(Dropout(0.3))
model_AM_ANN.add(BatchNormalization())
model_AM_ANN.add(Dense(128, activation='relu'))
model_AM_ANN.add(Dropout(0.3))
model_AM_ANN.add(BatchNormalization())
model_AM_ANN.add(Dense(64, activation='relu'))
model_AM_ANN.add(Dropout(0.3))
model_AM_ANN.add(BatchNormalization())
model_AM_ANN.add(Dense(1, activation='sigmoid')) 

model_AM_ANN.compile(loss='binary_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

es = EarlyStopping(monitor='val_accuracy', 
                   mode='max', 
                   patience=10,  #Stop the model training if the validation accuracy doesnt increase in 10 Epochs
                   restore_best_weights=True)

NN_Classifier_AM = model_AM_ANN.fit(X_train_AM_ANN,y_train_AM_ANN,
                          batch_size=1000,
                          epochs=1000,
                          callbacks =[es],
                          verbose=2,
                          validation_data=(X_test_AM_ANN, y_test_AM_ANN))
                          

y_pred=(model_AM_ANN.predict(X_test_AM_ANN) > 0.5).astype(int)
acc = accuracy_score(y_test_AM_ANN, y_pred)*100
#print('Accuracy Score: ', '{:.2f}%'.format(acc))

testingDataframe.loc[-1] = ["Bank normalizovan","Label Encoder","ANN",0,  0, '{:.2f}%'.format(acc)]
testingDataframe.index = testingDataframe.index + 1
testing = testingDataframe.sort_index()
testing

In [ ]:
# ANN - OneHot Encoding - Airline normalized

X_Airline_BM_ANN = normalised_airline_oh
y_Airline_BM_ANN = airlineDataset_OneHotEncoding['satisfaction']

# Split the data into training and testing sets
X_train_AM_ANN, X_test_AM_ANN, y_train_AM_ANN, y_test_AM_ANN = train_test_split(X_Airline_BM_ANN, y_Airline_BM_ANN, test_size=0.2, random_state=42)

model_AM_ANN = Sequential()
model_AM_ANN.add(Dense(512, activation='relu', input_shape=(X_train_AM_ANN.shape[1],)))
model_AM_ANN.add(Dropout(0.3))
model_AM_ANN.add(Dense(256, activation='relu'))
model_AM_ANN.add(Dense(256, activation='relu'))
model_AM_ANN.add(Dropout(0.3))
model_AM_ANN.add(Dense(128, activation='relu'))
model_AM_ANN.add(Dense(1, activation='sigmoid')) 

model_AM_ANN.compile(loss='binary_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

es = EarlyStopping(monitor='val_accuracy', 
                   mode='max', 
                   patience=10,  #Stop the model training if the validation accuracy doesnt increase in 10 Epochs
                   restore_best_weights=True)

NN_Classifier_AM = model_AM_ANN.fit(X_train_AM_ANN,y_train_AM_ANN,
                          batch_size=1000,
                          epochs=1000,
                          callbacks =[es],
                          verbose=2,
                          validation_data=(X_test_AM_ANN, y_test_AM_ANN))
                          

y_pred=(model_AM_ANN.predict(X_test_AM_ANN) > 0.5).astype(int)
acc = accuracy_score(y_test_AM_ANN, y_pred)*100
#print('Accuracy Score: ', '{:.2f}%'.format(acc))

testingDataframe.loc[-1] = ["Airline normalizovan","One Hot","ANN",0,  0, '{:.2f}%'.format(acc)]
testingDataframe.index = testingDataframe.index + 1
testing = testingDataframe.sort_index()
testing

<div style="border-radius:10px;
            background-color:#D6EAF8;
            letter-spacing:0.5px; text-align: center;">
            
<h3 style="padding: 5px 0px; color:#2980B9; font-weight: bold; font-family: Arial, sans-serif;">
ANN nad Bank standardizovanim skupom podataka</h3>
</div>

In [ ]:
standardized_airline_mix.head()

In [ ]:
# ANN - Mixed Encoding - Airline standardized

X_Airline_BM_ANN_S = standardized_airline_mix.drop('satisfaction',axis=1)
y_Airline_BM_ANN_S = airlineDataset_MixEncoding['satisfaction']


# Split the data into training and testing sets
X_train_AM_ANN, X_test_AM_ANN, y_train_AM_ANN, y_test_AM_ANN = train_test_split(X_Airline_BM_ANN_S, y_Airline_BM_ANN_S, test_size=0.2)

model_AM_ANN_M = Sequential()
model_AM_ANN_M.add(Dense(512, activation='relu', input_shape=(X_train_AM_ANN.shape[1],)))
model_AM_ANN_M.add(Dropout(0.3))
model_AM_ANN_M.add(Dense(256, activation='relu'))
model_AM_ANN_M.add(Dense(256, activation='relu'))
model_AM_ANN_M.add(Dropout(0.3))
model_AM_ANN_M.add(Dense(128, activation='relu'))
model_AM_ANN_M.add(Dense(1, activation='sigmoid')) 

model_AM_ANN_M.compile(loss='binary_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

es = EarlyStopping(monitor='val_accuracy', 
                   mode='max', 
                   patience=10,  #Stop the model training if the validation accuracy doesnt increase in 10 Epochs
                   restore_best_weights=True)

NN_Classifier_AM_S = model_AM_ANN_M.fit(X_train_AM_ANN,y_train_AM_ANN,
                          batch_size=1000,
                          epochs=1000,
                          callbacks =[es],
                          verbose=2,
                          validation_data=(X_test_AM_ANN, y_test_AM_ANN))

y_pred=(model_AM_ANN_M.predict(X_test_AM_ANN) > 0.5).astype(int)
acc = accuracy_score(y_test_AM_ANN, y_pred)*100
#print('Accuracy Score: ', '{:.2f}%'.format(acc))

testingDataframe.loc[-1] = ["Airline Standaradizovan","MIX","ANN",0,  0, '{:.2f}%'.format(acc)]
testingDataframe.index = testingDataframe.index + 1
testing = testingDataframe.sort_index()
testing

In [ ]:
# ANN - Label Encoding - Airline standardized

X_Airline_BM_ANN_S_L = standardized_airline_le.drop('satisfaction',axis=1)
y_Airline_BM_ANN_S_L = airlineDataset_LabelEncoding['satisfaction']

# Split the data into training and testing sets
X_train_AM_ANN, X_test_AM_ANN, y_train_AM_ANN, y_test_AM_ANN = train_test_split(X_Airline_BM_ANN_S_L, y_Airline_BM_ANN_S_L, test_size=0.2, random_state=42)

model_AM_ANN = Sequential()
model_AM_ANN.add(Dense(512, activation='relu', input_shape=(X_train_AM_ANN.shape[1],)))
model_AM_ANN.add(Dropout(0.3))
model_AM_ANN.add(Dense(256, activation='relu'))
model_AM_ANN.add(Dense(256, activation='relu'))
model_AM_ANN.add(Dropout(0.3))
model_AM_ANN.add(Dense(128, activation='relu'))
model_AM_ANN.add(Dense(1, activation='sigmoid')) 

model_AM_ANN.compile(loss='binary_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

es = EarlyStopping(monitor='val_accuracy', 
                   mode='max', 
                   patience=10,  #Stop the model training if the validation accuracy doesnt increase in 10 Epochs
                   restore_best_weights=True)

NN_Classifier_AM_SL = model_AM_ANN.fit(X_train_AM_ANN,y_train_AM_ANN,
                          batch_size=1000,
                          epochs=1000,
                          callbacks =[es],
                          verbose=2,
                          validation_data=(X_test_AM_ANN, y_test_AM_ANN))
                          

y_pred=(model_AM_ANN.predict(X_test_AM_ANN) > 0.5).astype(int)
acc = accuracy_score(y_test_AM_ANN, y_pred)*100
#print('Accuracy Score: ', '{:.2f}%'.format(acc))

testingDataframe.loc[-1] = ["Airline Standardizovan","Label Encoder","ANN",0,  0, '{:.2f}%'.format(acc)]
testingDataframe.index = testingDataframe.index + 1
testing = testingDataframe.sort_index()
testing

In [ ]:
# ANN - OneHot Encoding - Airline standardized

X_Airline_BM_ANN_S_OH = standardized_airline_oh.drop('satisfaction',axis=1)
y_Airline_BM_ANN_S_OH = airlineDataset_OneHotEncoding['satisfaction']

# Split the data into training and testing sets
X_train_AM_ANN, X_test_AM_ANN, y_train_AM_ANN, y_test_AM_ANN = train_test_split(X_Airline_BM_ANN_S_OH, y_Airline_BM_ANN_S_OH, test_size=0.2, random_state=42)

model_AM_ANN_OH = Sequential()
model_AM_ANN_OH.add(Dense(512, activation='relu', input_shape=(X_train_AM_ANN.shape[1],)))
model_AM_ANN_OH.add(Dropout(0.3))
model_AM_ANN_OH.add(Dense(256, activation='relu'))
model_AM_ANN_OH.add(Dense(256, activation='relu'))
model_AM_ANN_OH.add(Dropout(0.3))
model_AM_ANN_OH.add(Dense(128, activation='relu'))
model_AM_ANN_OH.add(Dense(1, activation='sigmoid')) 

model_AM_ANN_OH.compile(loss='binary_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

es = EarlyStopping(monitor='val_accuracy', 
                   mode='max', 
                   patience=10,  #Stop the model training if the validation accuracy doesnt increase in 10 Epochs
                   restore_best_weights=True)

NN_Classifier_AM_SOH = model_AM_ANN_OH.fit(X_train_AM_ANN,y_train_AM_ANN,
                          batch_size=1000,
                          epochs=1000,
                          callbacks =[es],
                          verbose=2,
                          validation_data=(X_test_AM_ANN, y_test_AM_ANN))
                          

y_pred=(model_AM_ANN_OH.predict(X_test_AM_ANN) > 0.5).astype(int)
acc = accuracy_score(y_test_AM_ANN, y_pred)*100
#print('Accuracy Score: ', '{:.2f}%'.format(acc))

testingDataframe.loc[-1] = ["Airline Standardizovan","One Hot","ANN",0,  0, '{:.2f}%'.format(acc)]
testingDataframe.index = testingDataframe.index + 1
testing = testingDataframe.sort_index()
testing

<div style="border-radius:10px;
            background-color:#D6EAF8;
            letter-spacing:0.5px; text-align: center;">
            
<h3 style="padding: 5px 0px; color:#2980B9; font-weight: bold; font-family: Arial, sans-serif;">
ANN nad Bank standardizovanim skupom podataka</h3>
</div>

In [ ]:
# ANN - Mixed Encoding - Bank dataset
X_Bank_BM_ANN = standardised_bank_mix.drop(['y'], axis=1)
y_Bank_BM_ANN = bankDataset_MixEncoding['y']

# Split the data into training and testing sets
X_train_BM_ANN, X_test_BM_ANN, y_train_BM_ANN, y_test_BM_ANN = train_test_split(X_Bank_BM_ANN, y_Bank_BM_ANN, test_size=0.2, random_state=42)

model_BM_ANN = Sequential()
model_BM_ANN.add(Dense(512, activation='relu', input_shape=(X_train_BM_ANN.shape[1],)))
model_BM_ANN.add(Dropout(0.3))
model_BM_ANN.add(Dense(256, activation='relu'))
model_BM_ANN.add(Dense(256, activation='relu'))
model_BM_ANN.add(Dropout(0.3))
model_BM_ANN.add(Dense(128, activation='relu'))
model_BM_ANN.add(Dense(1, activation='sigmoid')) 

model_BM_ANN.compile(loss='binary_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

es = EarlyStopping(monitor='val_accuracy', 
                   mode='max', 
                   patience=10,  #Stop the model training if the validation accuracy doesnt increase in 10 Epochs
                   restore_best_weights=True)

NN_Classifier_BM = model_BM_ANN.fit(X_train_BM_ANN,y_train_BM_ANN,
                          batch_size=1000,
                          epochs=1000,
                          callbacks =[es],
                          verbose=2,
                          validation_data=(X_test_BM_ANN, y_test_BM_ANN))
                          

y_pred=(model_BM_ANN.predict(X_test_BM_ANN) > 0.5).astype(int)
acc = accuracy_score(y_test_BM_ANN, y_pred)*100
#print('Accuracy Score: ', '{:.2f}%'.format(acc))

testingDataframe.loc[-1] = ["Bank standardizovan","MIX","ANN",0,  0, '{:.2f}%'.format(acc)]
testingDataframe.index = testingDataframe.index + 1
testing = testingDataframe.sort_index()
testing

In [ ]:
# ANN - Label Encoding -  Bank dataset
X_Bank_BLE_ANN = standardised_bank_le.drop(['y'], axis=1)
y_Bank_BLE_ANN = bankDataset_LabelEncoding['y']

# Split the data into training and testing sets
X_train_BLE_ANN, X_test_BLE_ANN, y_train_BLE_ANN, y_test_BLE_ANN = train_test_split(X_Bank_BLE_ANN, y_Bank_BLE_ANN, test_size=0.2, random_state=42)

model_BLE_ANN = Sequential()
model_BLE_ANN.add(Dense(512, activation='relu', input_shape=(X_train_BLE_ANN.shape[1],)))
model_BLE_ANN.add(Dropout(0.3))
model_BLE_ANN.add(Dense(256, activation='relu'))
model_BLE_ANN.add(Dense(256, activation='relu'))
model_BLE_ANN.add(Dropout(0.3))
model_BLE_ANN.add(Dense(128, activation='relu'))
model_BLE_ANN.add(Dense(1, activation='sigmoid')) 

model_BLE_ANN.compile(loss='binary_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

es = EarlyStopping(monitor='val_accuracy', 
                   mode='max', 
                   patience=10,  #Stop the model training if the validation accuracy doesnt increase in 10 Epochs
                   restore_best_weights=True)

NN_Classifier_BLE = model_BLE_ANN.fit(X_train_BLE_ANN, y_train_BLE_ANN,
                          batch_size=1000,
                          epochs=1000,
                          callbacks =[es],
                          verbose=2,
                          validation_data=(X_test_BLE_ANN, y_test_BLE_ANN))


y_pred=(model_BLE_ANN.predict(X_test_BLE_ANN) > 0.5).astype(int)
acc = accuracy_score(y_test_BLE_ANN, y_pred)*100

testingDataframe.loc[-1] = ["Bank standardizovan","Label Encoder","ANN",0,  0,  '{:.2f}%'.format(acc)]
testingDataframe.index = testingDataframe.index + 1
testing = testingDataframe.sort_index()
testing

In [ ]:
# ANN - One Hot Encoding -  Bank dataset
X_Bank_BOH_ANN = standardised_bank_oh.drop(['y'], axis=1)
y_Bank_BOH_ANN = bankDataset_OneHotEncoding['y']

# Split the data into training and testing sets
X_train_BOH_ANN, X_test_BOH_ANN, y_train_BOH_ANN, y_test_BOH_ANN = train_test_split(X_Bank_BOH_ANN, y_Bank_BOH_ANN, test_size=0.2, random_state=42)

model_BOH_ANN = Sequential()
model_BOH_ANN.add(Dense(512, activation='relu', input_shape=(X_train_BOH_ANN.shape[1],)))
model_BOH_ANN.add(Dropout(0.3))
model_BOH_ANN.add(Dense(256, activation='relu'))
model_BOH_ANN.add(Dense(256, activation='relu'))
model_BOH_ANN.add(Dropout(0.3))
model_BOH_ANN.add(Dense(128, activation='relu'))
model_BOH_ANN.add(Dense(1, activation='sigmoid')) 

model_BOH_ANN.compile(loss='binary_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

es = EarlyStopping(monitor='val_accuracy', 
                   mode='max', 
                   patience=10,  #Stop the model training if the validation accuracy doesnt increase in 10 Epochs
                   restore_best_weights=True)

NN_Classifier_BOH = model_BOH_ANN.fit(X_train_BOH_ANN, y_train_BOH_ANN,
                          batch_size=1000,
                          epochs=1000,
                          callbacks =[es],
                          verbose=2,
                          validation_data=(X_test_BOH_ANN, y_test_BOH_ANN))


y_pred=(model_BOH_ANN.predict(X_test_BOH_ANN) > 0.5).astype(int)
acc = accuracy_score(y_test_BOH_ANN, y_pred)*100

testingDataframe.loc[-1] = ["Bank standardizovan","One Hot","ANN",0,  0,  '{:.2f}%'.format(acc)]
testingDataframe.index = testingDataframe.index + 1
testing = testingDataframe.sort_index()
testing

<div style="border-radius:10px;
            background-color:#D6EAF8;
            letter-spacing:0.5px; text-align: center;">
            
<h3 style="padding: 5px 0px; color:#2980B9; font-weight: bold; font-family: Arial, sans-serif;">
ANN nad Bank normalizovanim skupom podataka</h3>
</div>

In [ ]:
# ANN - Mixed Encoding - Bank dataset
X_Bank_BM_ANN = normalised_bank_mix.drop('y',axis=1)
y_Bank_BM_ANN = bankDataset_MixEncoding['y']

# Split the data into training and testing sets
X_train_BM_ANN, X_test_BM_ANN, y_train_BM_ANN, y_test_BM_ANN = train_test_split(X_Bank_BM_ANN, y_Bank_BM_ANN, test_size=0.2, random_state=42)

model_BM_ANN = Sequential()
model_BM_ANN.add(Dense(512, activation='relu', input_shape=(X_train_BM_ANN.shape[1],)))
model_BM_ANN.add(Dropout(0.3))
model_BM_ANN.add(Dense(256, activation='relu'))
model_BM_ANN.add(Dense(256, activation='relu'))
model_BM_ANN.add(Dropout(0.3))
model_BM_ANN.add(Dense(128, activation='relu'))
model_BM_ANN.add(Dense(1, activation='sigmoid')) 

model_BM_ANN.compile(loss='binary_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

es = EarlyStopping(monitor='val_accuracy', 
                   mode='max', 
                   patience=10,  #Stop the model training if the validation accuracy doesnt increase in 10 Epochs
                   restore_best_weights=True)

NN_Classifier_BM = model_BM_ANN.fit(X_train_BM_ANN,y_train_BM_ANN,
                          batch_size=1000,
                          epochs=1000,
                          callbacks =[es],
                          verbose=2,
                          validation_data=(X_test_BM_ANN, y_test_BM_ANN))
                          

y_pred=(model_BM_ANN.predict(X_test_BM_ANN) > 0.5).astype(int)
acc = accuracy_score(y_test_BM_ANN, y_pred)*100
#print('Accuracy Score: ', '{:.2f}%'.format(acc))

testingDataframe.loc[-1] = ["Bank normalizovan","MIX","ANN",0,  0, '{:.2f}%'.format(acc)]
testingDataframe.index = testingDataframe.index + 1
testing = testingDataframe.sort_index()
testing

In [ ]:
# ANN - Label Encoding -  Bank dataset
X_Bank_BLE_ANN = normalised_bank_le.drop('y',axis=1)
y_Bank_BLE_ANN = bankDataset_LabelEncoding['y']

# Split the data into training and testing sets
X_train_BLE_ANN, X_test_BLE_ANN, y_train_BLE_ANN, y_test_BLE_ANN = train_test_split(X_Bank_BLE_ANN, y_Bank_BLE_ANN, test_size=0.2, random_state=42)

model_BLE_ANN = Sequential()
model_BLE_ANN.add(Dense(512, activation='relu', input_shape=(X_train_BLE_ANN.shape[1],)))
model_BLE_ANN.add(Dropout(0.3))
model_BLE_ANN.add(Dense(256, activation='relu'))
model_BLE_ANN.add(Dense(256, activation='relu'))
model_BLE_ANN.add(Dropout(0.3))
model_BLE_ANN.add(Dense(128, activation='relu'))
model_BLE_ANN.add(Dense(1, activation='sigmoid')) 

model_BLE_ANN.compile(loss='binary_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

es = EarlyStopping(monitor='val_accuracy', 
                   mode='max', 
                   patience=10,  #Stop the model training if the validation accuracy doesnt increase in 10 Epochs
                   restore_best_weights=True)

NN_Classifier_BLE = model_BLE_ANN.fit(X_train_BLE_ANN, y_train_BLE_ANN,
                          batch_size=1000,
                          epochs=1000,
                          callbacks =[es],
                          verbose=2,
                          validation_data=(X_test_BLE_ANN, y_test_BLE_ANN))


y_pred=(model_BLE_ANN.predict(X_test_BLE_ANN) > 0.5).astype(int)
acc = accuracy_score(y_test_BLE_ANN, y_pred)*100

testingDataframe.loc[-1] = ["Bank normalizovan","Label Encoder","ANN",0,  0,  '{:.2f}%'.format(acc)]
testingDataframe.index = testingDataframe.index + 1
testing = testingDataframe.sort_index()
testing

In [ ]:
# ANN - One Hot Encoding -  Bank dataset
X_Bank_BOH_ANN = normalised_bank_oh.drop('y',axis=1)
y_Bank_BOH_ANN = bankDataset_OneHotEncoding['y']

# Split the data into training and testing sets
X_train_BOH_ANN, X_test_BOH_ANN, y_train_BOH_ANN, y_test_BOH_ANN = train_test_split(X_Bank_BOH_ANN, y_Bank_BOH_ANN, test_size=0.2, random_state=42)

model_BOH_ANN = Sequential()
model_BOH_ANN.add(Dense(512, activation='relu', input_shape=(X_train_BOH_ANN.shape[1],)))
model_BOH_ANN.add(Dropout(0.3))
model_BOH_ANN.add(Dense(256, activation='relu'))
model_BOH_ANN.add(Dense(256, activation='relu'))
model_BOH_ANN.add(Dropout(0.3))
model_BOH_ANN.add(Dense(128, activation='relu'))
model_BOH_ANN.add(Dense(1, activation='sigmoid')) 

model_BOH_ANN.compile(loss='binary_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

es = EarlyStopping(monitor='val_accuracy', 
                   mode='max', 
                   patience=10,  #Stop the model training if the validation accuracy doesnt increase in 10 Epochs
                   restore_best_weights=True)

NN_Classifier_BOH = model_BOH_ANN.fit(X_train_BOH_ANN, y_train_BOH_ANN,
                          batch_size=1000,
                          epochs=1000,
                          callbacks =[es],
                          verbose=2,
                          validation_data=(X_test_BOH_ANN, y_test_BOH_ANN))


y_pred=(model_BOH_ANN.predict(X_test_BOH_ANN) > 0.5).astype(int)
acc = accuracy_score(y_test_BOH_ANN, y_pred)*100

testingDataframe.loc[-1] = ["Bank normalizovan","One Hot","ANN",0,  0,  '{:.2f}%'.format(acc)]
testingDataframe.index = testingDataframe.index + 1
testing = testingDataframe.sort_index()
testing

Poredjenje Modela

In [ ]:
testing = testingDataframe[testingDataframe['Algoritam']== 'ANN']
testing = testing.sort_values(by=['Accuracy'],ascending=False)
testing

In [ ]:
testing = testingDataframe[testingDataframe['Algoritam']== 'SVM']
testing = testing.sort_values(by=['Accuracy'],ascending=False)
testing

In [ ]:
testing = testingDataframe[testingDataframe['Algoritam']== 'Random forest']
testing = testing.sort_values(by=['Accuracy'],ascending=False)
testing

In [ ]:
testing = testingDataframe[testingDataframe['Algoritam']== 'Naive Bayes']
testing = testing.sort_values(by=['Accuracy'],ascending=False)
testing

<div style="border-radius:10px;
            background-color:#D6EAF8;
            letter-spacing:0.5px; text-align: center;">
            
<h2 style="padding: 5px 0px; color:#2980B9; font-weight: bold; font-family: Arial, sans-serif;">
Rukovanje ekstremnih vrednosti</h2>
</div>

In [ ]:
# Kopiranje dataseta za handlovanje outliera

normalised_bank_mix_ro = normalised_bank_mix.copy()
normalised_bank_mix_io = normalised_bank_mix.copy()


In [ ]:
# Funkcija za crtanje Box Plota
def BoxPlot(df):
    plt.figure(figsize=(5, 2))
    sns.boxplot(df)
    plt.show()

In [ ]:
# Funkcija za crtanje Histogram plota
def HistPlot(df):
    plt.figure(figsize=(5,2))
    plt.hist(df)
    plt.show()

In [ ]:
# Crtanje Box Plot-ova
for col in checkOutliers_bank:
    BoxPlot(normalised_bank_mix[col])



In [ ]:
# Crtanje Hist Plot-ova
for col in checkOutliers_bank:
    HistPlot(normalised_bank_mix[col])

In [ ]:
outliers_Bank = ['age','duration','campaign', 'cons.conf.idx']

In [ ]:
# Funckija za brisanje outliera
def drop_outliers(df,fieldname):
    iqr = 1.5*(np.percentile(df[fieldname],75) - np.percentile(df[fieldname],25))
    df.drop(df[df[fieldname] > (iqr + np.percentile(df[fieldname],75))].index,inplace=True)
    df.drop(df[df[fieldname] < ( np.percentile(df[fieldname],25) - iqr)].index,inplace=True)


In [ ]:
# Funckija za imputaciju outliera
def ImputateOutlier(df):
    q1 = df.quantile(0.25)
    q3 = df.quantile(0.75)
    iqr = q3 - q1
    LT = q1 - 1.5*iqr
    UT = q3 + 1.5*iqr
    df = df.clip(lower=LT, upper=UT)
    return df



In [ ]:
# Imputacija outliera
for col in outliers_Bank:
    ImputateOutlier(normalised_bank_mix_io[col])

In [ ]:
# Brisanje outliera
for col in outliers_Bank:
    drop_outliers(normalised_bank_mix_ro,col)

In [ ]:
# Crtanje Box Plot-ova nakon imputacije
for col in checkOutliers_bank:
    BoxPlot(normalised_bank_mix_io[col])

In [ ]:
# Crtanje Box Plot-ova nakon brisanja
for col in checkOutliers_bank:
    BoxPlot(normalised_bank_mix_ro[col])

In [ ]:
# ANN - Mixed Encoding - Bank dataset - Without outliers
X_Bank_BM_ANN = normalised_bank_mix_ro.drop('y', axis=1)
y_Bank_BM_ANN = normalised_bank_mix_ro['y']

# Split the data into training and testing sets
X_train_BM_ANN, X_test_BM_ANN, y_train_BM_ANN, y_test_BM_ANN = train_test_split(X_Bank_BM_ANN, y_Bank_BM_ANN, test_size=0.2, random_state=42)

model_BM_ANN = Sequential()
model_BM_ANN.add(Dense(512, activation='relu', input_shape=(X_train_BM_ANN.shape[1],)))
model_BM_ANN.add(Dropout(0.3))
model_BM_ANN.add(Dense(256, activation='relu'))
model_BM_ANN.add(Dense(256, activation='relu'))
model_BM_ANN.add(Dropout(0.3))
model_BM_ANN.add(Dense(128, activation='relu'))
model_BM_ANN.add(Dense(1, activation='sigmoid')) 

model_BM_ANN.compile(loss='binary_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

es = EarlyStopping(monitor='val_accuracy', 
                   mode='max', 
                   patience=10,  #Stop the model training if the validation accuracy doesnt increase in 10 Epochs
                   restore_best_weights=True)

NN_Classifier_BM = model_BM_ANN.fit(X_train_BM_ANN,y_train_BM_ANN,
                          batch_size=1000,
                          epochs=1000,
                          callbacks =[es],
                          verbose=2,
                          validation_data=(X_test_BM_ANN, y_test_BM_ANN))
                          

y_pred=(model_BM_ANN.predict(X_test_BM_ANN) > 0.5).astype(int)
acc = accuracy_score(y_test_BM_ANN, y_pred)*100
#print('Accuracy Score: ', '{:.2f}%'.format(acc))

testingDataframe.loc[-1] = ["Bank normalizovan w/ Outliers","MIX","ANN",0,  0, '{:.2f}%'.format(acc)]
testingDataframe.index = testingDataframe.index + 1
testing = testingDataframe.sort_index()
testing

In [ ]:
# ANN - Mixed Encoding - Bank dataset - Without outliers
X_Bank_BM_ANN = normalised_bank_mix_io.drop('y', axis=1)
y_Bank_BM_ANN = normalised_bank_mix_io['y']

# Split the data into training and testing sets
X_train_BM_ANN, X_test_BM_ANN, y_train_BM_ANN, y_test_BM_ANN = train_test_split(X_Bank_BM_ANN, y_Bank_BM_ANN, test_size=0.2, random_state=42)

model_BM_ANN = Sequential()
model_BM_ANN.add(Dense(512, activation='relu', input_shape=(X_train_BM_ANN.shape[1],)))
model_BM_ANN.add(Dropout(0.3))
model_BM_ANN.add(Dense(256, activation='relu'))
model_BM_ANN.add(Dense(256, activation='relu'))
model_BM_ANN.add(Dropout(0.3))
model_BM_ANN.add(Dense(128, activation='relu'))
model_BM_ANN.add(Dense(1, activation='sigmoid')) 

model_BM_ANN.compile(loss='binary_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

es = EarlyStopping(monitor='val_accuracy', 
                   mode='max', 
                   patience=10,  #Stop the model training if the validation accuracy doesnt increase in 10 Epochs
                   restore_best_weights=True)

NN_Classifier_BM = model_BM_ANN.fit(X_train_BM_ANN,y_train_BM_ANN,
                          batch_size=1000,
                          epochs=1000,
                          callbacks =[es],
                          verbose=2,
                          validation_data=(X_test_BM_ANN, y_test_BM_ANN))
                          

y_pred=(model_BM_ANN.predict(X_test_BM_ANN) > 0.5).astype(int)
acc = accuracy_score(y_test_BM_ANN, y_pred)*100
#print('Accuracy Score: ', '{:.2f}%'.format(acc))

testingDataframe.loc[-1] = ["Bank normalizovan Imputated Outliers","MIX","ANN",0,  0, '{:.2f}%'.format(acc)]
testingDataframe.index = testingDataframe.index + 1
testing = testingDataframe.sort_index()
testing

<div style="border-radius:10px;
            background-color:#D6EAF8;
            letter-spacing:0.5px; text-align: center;">
            
<h2 style="padding: 5px 0px; color:#2980B9; font-weight: bold; font-family: Arial, sans-serif;">
Diskretizacija podataka</h2>
</div>

<div style="border-radius:10px;
            background-color:#D6EAF8;
            letter-spacing:0.5px; text-align: center;">
            
<h3 style="padding: 5px 0px; color:#2980B9; font-weight: bold; font-family: Arial, sans-serif;">
Airline dataset</h3>
</div>

In [ ]:
airlineDataset

In [ ]:
airlineDataset.describe()

In [ ]:
#Columns for discretization
    # Age, Flight Distance, Grades, Delay

In [ ]:
airlineDataset.dtypes

In [ ]:
airline_discretization_columns = ['Age','Flight Distance','Departure Delay in Minutes','Arrival Delay in Minutes']
airline_rating_columns = airlineDataset.columns[7:21]
airline_rating_columns_names = list(airline_rating_columns)
airline_rating_columns_names

In [ ]:
from sklearn.preprocessing import KBinsDiscretizer

# Select the variable to discretize
age = airlineDataset[['Age']]

# Create a KBinsDiscretizer object with 5 bins
kbins = KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='uniform')

# Fit and transform the age variable
age_binned = kbins.fit_transform(age)

# Print the resulting binned values
airlineDataset['Age'] = age_binned

airlineDataset

In [ ]:
airlineDataset['Arrival Delay in Minutes'].value_counts()

In [ ]:
# Replace values of 0 with 'On time' and values of 1 or more with 'Delayed'
airlineDataset['Arrival Delay in Minutes'] = airlineDataset['Arrival Delay in Minutes'].replace(0, 'On time').replace(range(1, int(airlineDataset['Arrival Delay in Minutes'].max()) + 1), 'Delayed')

# Print the resulting value counts
print(airlineDataset['Arrival Delay in Minutes'].value_counts())


In [ ]:
# Replace values of 0 with 'On time' and values of 1 or more with 'Delayed'
airlineDataset['Departure Delay in Minutes'] = airlineDataset['Departure Delay in Minutes'].replace(0, 'On time').replace(range(1, airlineDataset['Departure Delay in Minutes'].max() + 1), 'Delayed')

# Print the resulting value counts
print(airlineDataset['Departure Delay in Minutes'].value_counts())

In [ ]:
labele=['Unsatisfied','Neutral','Satisfied']
binovi = [0,1,3,5]
for label in airline_rating_columns_names:
    airlineDataset[label] = pd.cut(airlineDataset[label],binovi,labele)
airlineDataset

In [ ]:
for label in airline_rating_columns_names:
    airlineDataset[label] = pd.cut(airlineDataset[label],bins=3,labels=['Unsatisfied','Neutral','Satisfied'])
airlineDataset

In [ ]:

# Odabir atributa
flight_distance = airlineDataset[['Flight Distance']]

# Kreiranje KBinsDiscretizer objekta s 3 kategorije
kbins = KBinsDiscretizer(n_bins=3, encode='ordinal', strategy='kmeans')

# Primjena diskretizacije na atribut Flight Distance
flight_distance_binned = kbins.fit_transform(flight_distance)

# Dodavanje kategorija u dataset
airlineDataset['Flight Distance'] = flight_distance_binned



In [ ]:
for column in airlineDataset.columns:
    print(airlineDataset[column].value_counts())

<div style="border-radius:10px;
            background-color:#D6EAF8;
            letter-spacing:0.5px; text-align: center;">
            
<h2 style="padding: 5px 0px; color:#2980B9; font-weight: bold; font-family: Arial, sans-serif;">
Promena raspodele podataka</h2>
</div>

In [ ]:
vehiclesDataset = vehiclesDataset.drop(['angle', 'pos', 'lane', 'slope', 'signals', 'posLat'], axis=1)\
       .rename(columns={'x': 'longitude', 'y': 'latitude'})\
       .assign(speed_kmh=lambda x: round(x['speed'] * 3.6, 2))\
       .drop(['speed'],axis=1)


In [ ]:
vehiclesDataset.dtypes

In [ ]:
vehiclesDataset.head(10)

In [ ]:
vehiclesDataset = vehiclesDataset[['id','timestep','type','latitude','longitude','speed_kmh','acceleration','distance','odometer']]

In [ ]:
vehiclesDataset.head(10)

In [ ]:
vehiclesDataset['type'] = vehiclesDataset['type'].replace({'bus_bus':'bus', 'veh_passenger':'car'})


In [ ]:
vehiclesDataset['timestep'].nunique()

In [ ]:
vehiclesDataset['time_of_day'] = vehiclesDataset['timestep'].apply(lambda x: 'morning' if x < 61 else ('afternoon' if 61 <= x <= 899 else 'evening'))
vehiclesDataset.drop(['timestep','id'],axis=1)

In [ ]:
vehiclesDataset['time_of_day'].value_counts()

In [ ]:
morning_df = vehiclesDataset[vehiclesDataset['time_of_day'] == 'morning']


In [ ]:
afternoon_df = vehiclesDataset[vehiclesDataset['time_of_day'] == 'afternoon']


In [ ]:
evening_df = vehiclesDataset[vehiclesDataset['time_of_day'] == 'evening']


In [ ]:
morning_df.describe()

In [ ]:
afternoon_df.describe()

In [ ]:
evening_df.describe()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Učitavanje podataka i računanje statistika

stats = evening_df.describe()

# Kreiranje grafikona
fig, ax = plt.subplots(figsize=(10, 5))
stats.plot(kind='bar', ax=ax)

# Podešavanje naslova i oznaka osa
ax.set_title('Statistike')
ax.set_xlabel('Statistika')
ax.set_ylabel('Vrednost')

# Prikazivanje grafikona
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Prikaz histograma brzine za svaki skup
fig, ax = plt.subplots(figsize=(10, 6))
ax.hist(evening_df['speed_kmh'], bins=50, alpha=0.5, label='Evening')
ax.hist(afternoon_df['speed_kmh'], bins=50, alpha=0.5, label='Afternoon')

# Dodavanje traka za prikaz statističkih mjera za svaki skup
evening_stats = evening_df.describe()['speed_kmh']
afternoon_stats = afternoon_df.describe()['speed_kmh']
ax.axvline(evening_stats['mean'], color='blue', linestyle='--', label='Evening Mean')
ax.axvline(afternoon_stats['mean'], color='black', linestyle='--', label='Afternoon Mean')
ax.axvline(evening_stats['std']+evening_stats['mean'], color='blue', linestyle=':', label='Evening Std')
ax.axvline(afternoon_stats['std']+afternoon_stats['mean'], color='orange', linestyle=':', label='Afternoon Std')
ax.axvline(evening_stats['min'], color='blue', linestyle='-.', label='Evening Min')
ax.axvline(afternoon_stats['min'], color='black', linestyle='-.', label='Afternoon Min')
ax.axvline(evening_stats['max'], color='blue', linestyle='-.', label='Evening Max')
ax.axvline(afternoon_stats['max'], color='black', linestyle='-.', label='Afternoon Max')

# Postavljanje naziva i legendi za grafikon
ax.set_title('Comparison of Speed Distribution and Statistics between Evening and Afternoon')
ax.set_xlabel('Speed (km/h)')
ax.set_ylabel('Frequency')
ax.legend()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
def prikazi_mere(column):
    # Prikaz histograma brzine za svaki skup
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.hist(evening_df[column], bins=50, alpha=0.5, label='Evening')
    ax.hist(afternoon_df[column], bins=50, alpha=0.5, label='Afternoon')
    ax.hist(morning_df[column], bins=50, alpha=0.5, label='Morning')

    # Dodavanje traka za prikaz statističkih mjera za svaki skup
    evening_stats = evening_df.describe()[column]
    afternoon_stats = afternoon_df.describe()[column]
    morning_stats = morning_df.describe()[column]

    ax.axvline(evening_stats['mean'], color='blue', linestyle='--', label='Evening Mean')
    ax.axvline(afternoon_stats['mean'], color='black', linestyle='--', label='Afternoon Mean')
    ax.axvline(morning_stats['mean'], color='green', linestyle='--', label='Morning Mean')

    ax.axvline(evening_stats['std']+evening_stats['mean'], color='blue', linestyle=':', label='Evening Std')
    ax.axvline(afternoon_stats['std']+afternoon_stats['mean'], color='orange', linestyle=':', label='Afternoon Std')
    ax.axvline(morning_stats['std']+morning_stats['mean'], color='purple', linestyle=':', label='Morning Std')

    ax.axvline(evening_stats['min'], color='blue', linestyle='-.', label='Evening Min')
    ax.axvline(afternoon_stats['min'], color='black', linestyle='-.', label='Afternoon Min')
    ax.axvline(morning_stats['min'], color='green', linestyle='-.', label='Morning Min')

    ax.axvline(evening_stats['max'], color='blue', linestyle='-.', label='Evening Max')
    ax.axvline(afternoon_stats['max'], color='black', linestyle='-.', label='Afternoon Max')
    ax.axvline(morning_stats['max'], color='green', linestyle='-.', label='Morning Max')

    # Postavljanje naziva i legendi za grafikon
    ax.set_title('Comparison of Distribution and Statistics between Evening, Morning and Afternoon')
    ax.set_xlabel(column)
    ax.set_ylabel('Frequency')
    ax.legend()
    plt.show()


In [ ]:
num_cols = vehiclesDataset.select_dtypes(include=['int', 'float']).columns.tolist()


In [ ]:
for col in num_cols:
    prikazi_mere(col)